<a href="https://colab.research.google.com/github/ignyz/BBP_Ignas_Narusis_IFK7/blob/main/Kodo_sutvarkymas_pateikimui_AmbersCorvayorWithUseCase_Modifikavimas_su_modelio_issaugojimu_ipynb%2C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2 
import numpy as np
import os
from matplotlib import pyplot as plt
import pickle
 
##############################################
#         Kodo importavimo vieta
 
import natsort 
from datetime import datetime
import os
import shutil

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, siluetoTaskai

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import matplotlib.style as style
 
##############################################

import json
import ast

import time
import pathlib
import pandas as pd
import os.path
import os, psutil
import numpy as np
import matplotlib.pyplot as plt
import math
import itertools
import gc
import cv2 
 
import skimage
import sklearn
from sklearn.decomposition import PCA 
from sklearn.feature_extraction import nuotrauka
from skimage.segmentation import chan_vese
from skimage.morphology import closing, square, disk
from skimage.io import imread
from skimage.filters import rank, threshold_otsu
from skimage.feature import canny
from skimage.color import rgb2grey
from skimage.color import rgb2gray
from sklearn.ensemble import RandomForestClassifier
from skimage import exposure as hist, data, img_as_float
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn.svm import SVR

import scipy.stats as stats
from scipy.spatial.distance import cdist
from scipy.spatial import distance
from scipy.ndimage.interpolation import rotate
from scipy import spatial
from scipy import ndimage as ndi 
from scipy import ndimage
from matplotlib import pyplot as plt
from glob import glob
from PIL import ImageTk
from PIL import Image, TiffImagePlugin
from PIL import Image as Img
from PIL import Image
from IPython.display import Image, display

import json

##############################################
#   SOM bibliotekos įdiegimas

!pip install sklearn-som
from sklearn_som.som import SOM
from sklearn import datasets
 
# Google colab bibliotekų įkėlimas, skirtos duomenų nuskaitymui, apdorojimui, išsaugojimui 
from google.colab.patches import cv2_imshow
from google.colab import files
from google.colab import drive
import sys
import platform
import seaborn as sn
 
drive.mount('/content/gdrive')

sys.path.append('/content/gdrive/My Drive/Bakalauras/Data/ambers')
print(sys.version)
platform.python_version()
sys.version_info



#           Globalūs kintmieji 

# 8 - sienų nurodymas, grandinės kodui.
kodoSarasas = [5,6,7,4,8,0,3,2,1]

# Pasirinkimas atspausdinti tarpinius rezultatų kintamuosius ir rezultatus
arSektiIrasus = 0
#         Nuotraukų apdorojimų metodai


def kombinacijos(s):
  if len(s) == 2:
    yield [s]
  for x in s[1:]:
    pirmaDalis = ''.join((s[0], x))
    likusiDalis = ''.join(c for c in s if c not in pirmaDalis)
    for kombinacija in kombinacijos(likusiDalis):
      yield [pirmaDalis] + kombinacija

def sumazintiNuotraukosMasteli(nuotrauka, plotis=None, aukstis=None, inter=cv2.INTER_AREA):
    dimensija = None
    (aukst, plot) = nuotrauka.shape[:2]
 
    if plotis is None and aukstis is None:
        return nuotrauka
    if plotis is None:
        r = aukstis / float(aukst)
        dimensija = (int(plot * r), aukstis)
    else:
        r = plotis / float(plot)
        dimensija = (plotis, int(aukst * r))
 
    return cv2.resize(nuotrauka, dimensija, interpolation=inter)


def maziausiaDezute(a): # nukerpame nereikalingas gintaro dalis.
            r = a.any(1)
            if r.any():
                m,n = a.shape
                c = a.any(0)
                rezultatas = a[r.argmax():m-r[::-1].argmax(), c.argmax():n-c[::-1].argmax()]
            else:
                rezultatas = np.empty((0,0),dtype=bool)
            return rezultatas

def taskavimas(a,x,y): # 
    r = a
    r[x][y] = 0;
    c,v = r.shape
    if c > x+1:
      r[x+1][y] = 255
    if c > x-1 and x-1 >= 0:
      r[x-1][y] = 255
    if c > x-2 and x-2 >= 0:
      r[x-1][y] = 0
    return r


def vertikalusApvertimas(nuotrauka):
  """
  Pilkojo vaizdo atvaizdo pasukimo metodas, atsižvelgiant į jo pagrindinę ašį. 
  """
  
  # Gauname mus dominančias objekto koordinates
  X = np.array(np.where(nuotrauka > 0)).T
  if len(X) > 0:

    # PCA apskaiciavimas kampo pagal pagrindines asis
    rezultatasPCA = PCA(n_components=2).fit(X)
    kampas = np.arctan2(*rezultatasPCA.components_[0])
    
    # Apsukame nuotrauką pagal apskaičiuotą kampą.
    pasuktaNuotrauka = rotate(nuotrauka,kampas/math.pi*180-90)

    pasuktaNuotrauka = nezinomuReiksmiuPasalinimas(pasuktaNuotrauka)
    return pasuktaNuotrauka
  else:
    return nuotrauka


def perziuretiDvejetaineNuotrauka(aplankalas="/content/gdrive/My Drive/Bakalauras/Data/image_from_binary"):
  
  sarasas = os.listdir(aplankalas)
  sarasas = sorted(sarasas, key=lambda a: int(a.split(".")[0].split("-")[0]),reverse=False )
  
  for failoPavadinimas in sarasas:
      failoPatikra1 = pathlib.Path(os.path.join(aplankalas,failoPavadinimas))
      
      if failoPatikra1.exists():
        print(failoPavadinimas)

        nuotrauka = cv2.imread(aplankalas+ "/" + failoPavadinimas, 2) 
        nuotrauka = cv2.resize(nuotrauka, (224, 224))
            
        nuotrauka[nuotrauka < 200] = 0
        nuotrauka[nuotrauka > 200] = 255

        plt.imshow(nuotrauka, interpolation='nearest')
        plt.show()
        
  print("done")

def nuotraukos_grotelelese(nuotraukos, eiles, kolonos):
  assert len(nuotraukos) == eiles*kolonos

  plot, aukst = nuotraukos[0].size
  groteles = Image.new('RGB', size=(kolonos*plot, eiles*aukst))
  groteles_plot, groteles_aukstis = groteles.size
  
  for i, nuotrauka in enumerate(nuotraukos):
      groteles.paste(nuotrauka, box=(i%kolonos*plot, i//kolonos*aukst))
  return groteles


# Dvejetainės nuotraukos savybių išgavimo metodai 

def centroido_apskaiciavimas(nuotrauka):

  # Konvertuojame nuotrauka į pilko formato nuotrauką
  spalva = 255
  
  # Konvertuoti pilkos formos nuotrauka i binarine
  # isrankioti nuotraukos momentus
  Momentai = cv2.moments(nuotrauka)

  # apskaičiuojame x ir y centro koordinates
  koordinate1, koordinate2 = ndimage.measurements.center_of_mass(nuotrauka)

  koordinateX = int(koordinate1)#int(M["m10"] / M["m00"])
  koordinateY = int(koordinate2)#int(M["m01"] / M["m00"])

  # ikelti teksta ir ji pazymeti
  cv2.circle(nuotrauka, (koordinateX, koordinateY), 5, (spalva, spalva, spalva), -1)
  cv2.putText(nuotrauka, "centroid", (koordinateX - 25, koordinateY - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (spalva, spalva, spalva), 2)

  # rodyti nuotrauka
  plt.imshow(nuotrauka, interpolation='nearest')
  plt.show()

  koordinateX = int(Momentai["m10"] / Momentai["m00"])
  koordinateY = int(Momentai["m01"] / Momentai["m00"])

  # ikelti ir pazymeti teksta
  cv2.circle(nuotrauka, (koordinateX, koordinateY), 5, (spalva, spalva, spalva), -1)
  cv2.putText(nuotrauka, "centroid", (koordinateX - 25, koordinateY - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (spalva, spalva, spalva), 2)

  plt.imshow(nuotrauka, interpolation='nearest')
  plt.show()

def masesCentras(nuotrauka):
  momentasX = 0
  momentasY = 0
  skaitliukas = 0

  for i in range(nuotrauka.shape[0]):
      for j in range(nuotrauka.shape[1]):
          if nuotrauka[i, j] >= 255:
              momentasX = momentasX + j
              momentasY = momentasY + i
              skaitliukas = skaitliukas + 1

  if skaitliukas > 0:
    centrasX = int(momentasX / skaitliukas)
    centrasY = int(momentasY / skaitliukas)
    return centrasX, centrasY
  else:
    return np.nan, np.nan

def uzkrautiNuotraukas(vietos,aplankalas):
    laikinas = []
    for vieta in vietos:
        vietosPatikrinimas = pathlib.Path(os.path.join(aplankalas,vieta))
        laikinas.append(imread(vietosPatikrinimas))
    return laikinas

def kombinacijosSeka(s, centroidai, pasirinkimas, pasirinkimas_c = 1):
  sarasas = []
  likusiDalis = ''.join(str(c) for c in range(len(s)))
  sasasoSuma = []
  centroiduSarasas = []

  kombinacijosR = list(kombinacijos(likusiDalis))
  for kombA in kombinacijosR:

    mazasSarasas = []
    mazasCentroiduSarasas = []
    sarasasS = 0

    for kombB in kombA:

      didziausiasSarasas = []
      laikinasCentroiduSarasas = []

      for kombC in kombB:

        didziausiasSarasas.append(s[int(c)])

        if len(centroidai) > 0:
          laikinasCentroiduSarasas.append(centroidai[int(c)])

      sarasasS += (abs(np.diff(didziausiasSarasas)))
      mazasSarasas.append(didziausiasSarasas)
      mazasCentroiduSarasas.append(laikinasCentroiduSarasas)

    sasasoSuma.append(float(sarasasS))
    sarasas.append(mazasSarasas)
    centroiduSarasas.append(mazasCentroiduSarasas)

  # gauname surusiuoto array pointerius, nerusiuojant pacio array
  surusuiuotosTaskuReiksmes = [i[0] for i in sorted(enumerate(sasasoSuma), key=lambda i: i[1])][0] 
  
  rezultatas = []
  for kombA in sarasas[surusuiuotosTaskuReiksmes]:
    if pasirinkimas == 0:      
      rezultatas.append(max(kombA))
    elif pasirinkimas == 1:
        rezultatas.append(sum(kombA)/len(kombA))
  
  if len(centroidai) > 0:
    centroiduRezultatai = []

    for kombA in centroiduSarasas[surusuiuotosTaskuReiksmes]:

      if pasirinkimas_c == 0:      
        centroiduRezultatai.append(max(kombA))
      elif pasirinkimas_c == 1:
          centroiduRezultatai.append(sum(kombA)/len(kombA))

    # print(details)  
    return rezultatas, sarasas[surusuiuotosTaskuReiksmes], centroiduRezultatai
  else:
    return rezultatas, sarasas[surusuiuotosTaskuReiksmes]


def nezinomuReiksmiuPasalinimas(nuotr):
  if nuotr is not None:
    nuotr[nuotr < 200] = 0
    nuotr[nuotr > 200] = 255
  return nuotr

def kaimynuKiekis(nuotraukuMatrica, i, j, W, H): 
    skaitliukas = 0; 
    if (i > 0 and nuotraukuMatrica[i - 1][j]): 
        skaitliukas+= 1
    if (j > 0 and nuotraukuMatrica[i][j - 1]): 
        skaitliukas+= 1
    if (i < W-1 and nuotraukuMatrica[i + 1][j]): 
        skaitliukas+= 1
    if (j < H-1 and nuotraukuMatrica[i][j + 1]): 
        skaitliukas+= 1
    return skaitliukas

# graziname perimetro suma 
def rastiPerimetra(matavimai,R,C): 
    perimetras = 0; 
    koordinates = []
    for i in range(0, R): 
        for j in range(0, C): 
            if (matavimai[i][j]): 
                Per  = kaimynuKiekis(matavimai, i, j,R,C)
                perimetras += 4 - Per
                if 4 - Per > 0:
                  koordinates.append([i,j])

    return perimetras, koordinates; 

def gintaroPerimetroAtvaizdavimas(a):
  x1 = (a[:,1:] != a[:,:-1])[1:,:] 
  x2 = (a[1:,:] != a[:-1,:])[:,1:] 
  y1 = x1.astype(int)
  y2 = x2.astype(int)
  Rxy = y1+y2
  return Rxy

def gintaroPerimetroPristatymasViesai(matavimai,W,H):
  koordinates = []
  perimetras = 0;
  for i in range(0, W): 
      for j in range(0, H): 
          if (matavimai[i][j]): 
              skaitliukas = 0; 
              if (i > 0 and matavimai[i - 1][j]): 
                  skaitliukas+= 1
              if (j > 0 and matavimai[i][j - 1]): 
                  skaitliukas+= 1
              if (i < W-1 and matavimai[i + 1][j]): 
                  skaitliukas+= 1
              if (j < H-1 and matavimai[i][j + 1]): 
                  skaitliukas+= 1
              Per  = 4 - count
              perimetras += Per
              if Per > 0:
                koordinates.append([i,j])
  return koordinates

def xyApgaubtasPlotas(xyFailas):
	plotas = spatial.ConvexHull(xyFailas, qhull_options="Qt")
	plotoIndeksai = plotas.vertices
	return plotoIndeksai


def apgaubtasPerimetas(a):
  '''
  # kampų radimas ir convex perimetras
  ''' 
  a = np.array(a)
  
  plotoIndeksai = xyApgaubtasPlotas(a)
  plotoIndeksaiXY = []

  for i in range(len(plotoIndeksai)):

    index = plotoIndeksai[i]
    value_y = a[index, 0].astype('float32')
    value_x = a[index, 1].astype('float32')
    plotoIndeksaiXY.append([value_y, value_x])
  
  plotoIndeksaiXY = np.array(plotoIndeksaiXY)
  atstumuSarasas = (plotoIndeksaiXY[:-1,0]-plotoIndeksaiXY[1:,0])**2 + (plotoIndeksaiXY[:-1,1]-plotoIndeksaiXY[1:,1])**2

  abgaubtasPerimetras  = np.sum(np.sqrt(atstumuSarasas)) 

  return plotoIndeksaiXY, len(plotoIndeksai),  abgaubtasPerimetras


def atstumas(p,q):
  return math.sqrt(sum((px - qx) ** 2.0 for px, qx in zip(p, q)))

def rodytiGintaroAsis(X,a1,a2,c1, c2, w_bounds,h_bounds ):
  at = np.transpose([a1,a2])
  plt.plot(X[0,:], X[1,:], 'k.')
  plt.plot(at[0],at[1], '-ok')
  plt.axis('equal')
  # plt.gca().invert_yaxis()

def pagindineAsis(am,X,c1, c2, w_bounds,h_bounds ):
  Xt = am #np.transpose(X)
  atstumai = distance.cdist(Xt, Xt, 'euclidean')

  # Randame didžiausią reikšmę iš 2D masyvo pažymėdami jį į indeksą.
  rezultatas = np.where(atstumai == np.amax(atstumai))

  koordinaciuSarasas = list(zip(rezultatas[0], rezultatas[1]))
  koord = koordinaciuSarasas[0]

  a1 = list(Xt[koord[0]])
  a2 = list(Xt[koord[1]])
  xy_atstumas = atstumas(Xt[koord[0]],Xt[koord[1]])
  
  if arSektiIrasus:
    rodytiGintaroAsis(X,a1,a2,c1, c2, w_bounds,h_bounds )

  return xy_atstumas, [a1,a2]
  
def pagrindinesAsiesKampas(aKoordinates):
  t1 = (aKoordinates[1][1]-aKoordinates[0][1])
  t2 = (aKoordinates[1][0]-aKoordinates[0][0])

  if t2 != 0:
    tn = t1/t2
    atanLaipsniai = math.degrees(math.atan(tn))
    return atanLaipsniai
  else: 
    return 0
    
def apgaubtoHullPlotas(pts):
    pts
    eilutes = np.hstack([pts,np.roll(pts,-1,axis=0)])
    plotas = 0.5*abs(sum(x1*y2-x2*y1 for x1,y1,x2,y2 in eilutes))
    return plotas

def centro_WH(x,y,a):
  xIlgis=0
  y_ilgis=0

  for i in range(a.shape[0]):
    if a[i][int(y)] > 0:
      y_ilgis += 1

  for i in range(a.shape[1]):
    if a[int(x)][i] > 0:
      xIlgis += 1
  return xIlgis, y_ilgis
  
def gintaroPlotas(a):
  pikseliai = cv2.countNonZero(a) # suskaiciuojame kiek yra uzpildyto nuotrauka pixeliu
  nuotraukosPlotas = a.shape[0] * a.shape[1] # nuotraukos plotis ir ilgis 
  plotoSantykis = (pikseliai / nuotraukosPlotas) * 100 # plotis gintaro nuotraukoje;
  
  # Atspausdiname paprastus rezultatus patikrai
  if arSektiIrasus:
    print('pikseliai', pikseliai)
    print('Image plotas', nuotraukosPlotas, a.shape[0], a.shape[1])
    print('plotas ratio', plotoSantykis)

  return pikseliai, plotoSantykis


def atspausdintiMatrica(a):
  print('\n'.join([''.join(['{:4}'.format(item) for item in row])  for row in a]))


def gintaroMazDidzAsys(perimetroKoordinates):
  perimetrasC = np.array(perimetroKoordinates)
  perimetrasCTransponuotas = np.transpose(perimetrasC)

  return perimetrasC, perimetrasCTransponuotas


def rastiKompaktiskuma(A, P): 
  # isoperimetric quotient on a plane: https://en.wikipedia.org/wiki/Isoperimetric_inequality
  # http://www.markedbyteachers.kombinacija/gcse/maths/to-investigate-the-isoperimetric-quotient-iq-of-plane-shapes-using-the-calculation-shown-below.html

  Q = 0;

  if P > 0:
    Q = ( 4 * math.pi * A) / (P**2)
  return Q
  
def koordinatesI2Dsarasa(P,plot,aukst):

  gintaroRibuMatrica = np.full((aukst + 1, plot + 1), False, dtype=bool)

  for k in P:
    x = k[0]
    y = k[1]
    gintaroRibuMatrica[y,x] = True

  # atspausdintiMatrica(gintaroRibuMatrica)

  return gintaroRibuMatrica

def rusiuotiXYPagalLaikrodzioRod(x, y): 

  Xnull = np.mean(x)
  Ynull = np.mean(y)

  r = np.sqrt((x-Xnull)**2 + (y-Ynull)**2)

  kampai = np.where((y-Ynull) > 0, np.arccos((x-Xnull)/r), 2*np.pi-np.arccos((x-Xnull)/r))

  mk = np.argsort(kampai)

  surusiuotasX = x[mk]
  surusiuotasY = y[mk]

  return surusiuotasX, surusiuotasY


def grandinelesKodas(x1, y1, x2, y2):   
  grandinelesRaktas = 3 * (x2 - x1 ) + (y2 - y1) + 4
  
  if grandinelesRaktas >= len(kodoSarasas) or  0 >= grandinelesRaktas :
    return 8
  else :
    return kodoSarasas[grandinelesRaktas]
  
def grandinelesSarasas(sarasasXY): 

  grandinelesSarasas = [] 
  
  for i in range(len(sarasasXY) - 1): 
      a = sarasasXY[i] 
      b = sarasasXY[i + 1] 
      grandinelesSarasas.append(grandinelesKodas(a[0], a[1], b[0], b[1])) 
  
  return grandinelesSarasas 

def ismatuotiConvexity(a,b):
  return a/b


def dydzioFormatas(b): # Išvestis skirta atspausdinti sunaudojamą ram kiekį.

  if b < 1000:
            return '%i' % b + 'B'

  elif 1000 <= b < 1000000:
      return '%.1f' % float(b/1000) + 'KB'

  elif 1000000 <= b < 1000000000:
      return '%.1f' % float(b/1000000) + 'MB'

  elif 1000000000 <= b < 1000000000000:
      return '%.1f' % float(b/1000000000) + 'GB'

  elif 1000000000000 <= b:
      return '%.1f' % float(b/1000000000000) + 'TB'



def nuskaitytiOriginaliaNuotrauka_KonvertuotiIOriginaliaApkarpyta(aplankalas , dvejetainiuNuotraukuVieta ):
  # Kodo dalis skirta nuskaityti nuotraukas, o tas nuotraukas sumazinti rezoliucija ir issaugoti binary formatu nuotraukoje, jeigu ta nuotrauka
  # egzistuoja jos darkartą nesaugos į binary nuotrauk1/formą kad sutaupytume laiko ir vietos size buvo sumažintas iki 360 pločio su Automatiniu aspect ratio

  testi = True
  try:
    sarasas = os.listdir(aplankalas)
  except FileNotFoundError:
    print("!! Folder was not found !!")
    testi = False
  if testi:
    sarasas = sorted( sarasas, key=lambda a: int(a.split(".")[0].split("-")[0]),reverse=False )
    for failoPavadinimas in sarasas:
        
        failoPatikra1 = pathlib.Path(os.path.join(aplankalas,failoPavadinimas))
        if failoPatikra1.exists():
          dvejetainioFailoPavadinimas = failoPavadinimas #.split(".")[0]  + ".bin"
          if not os.path.isdir(os.path.abspath(dvejetainiuNuotraukuVieta)):
            if not os.path.exists(dvejetainiuNuotraukuVieta):
              os.mkdir(dvejetainiuNuotraukuVieta)
              print("Creating new directory: ", dvejetainiuNuotraukuVieta)


          failoPatikra2 = pathlib.Path(os.path.join(dvejetainiuNuotraukuVieta, dvejetainioFailoPavadinimas) )
          print(failoPatikra2)
          
          if failoPatikra2.exists() != True :
            nuotrauka = cv2.imread(os.path.join(aplankalas,failoPavadinimas),0) 

            # patikrinimas jei None - tai reiskia blogai nusiskaitė, jei yra duomenys tada reiskiasi yra nuskaitytas
            if nuotrauka is not None:  
              ret, bw_nuotrauka = cv2.threshold(nuotrauka, 127, 255, cv2.THRESH_BINARY) 

              # Konvertuojame į binarinę formą 
              # # binarines formos array kuris bus panaudojamas duomenims ir ju analizei
              bw_nuotrauka_rs = sumazintiNuotraukosMasteli(bw_nuotrauka, plotis=360) 
              # Dydzio pakeitimas H: 480 W: 360          
              # dydi keiciama pagal ploti islaikant vienoda aspect ratio; 

              suliejimas = cv2.GaussianBlur(bw_nuotrauka_rs, (3,3), 0)
              thresh = cv2.threshold(suliejimas, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

              # Morph uzdarrymas ir invertavimas nuotraukos
              kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
              close = 255 - cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
              des = cv2.bitwise_not(close)

              contour,hier = cv2.findContours(des,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)

              for skaitliukas in contour:
                  cv2.drawContours(des,[skaitliukas],0,255,-1)

              bw_nuotrauka_rs = des
              kernel = np.ones((5,5),np.uint8)
              erosion = cv2.erode(bw_nuotrauka_rs,kernel,iterations = 1)

              QR_final = vertikalusApvertimas(erosion)
              QR_final = maziausiaDezute(QR_final)
              
              
              c1, c2 = ndimage.measurements.center_of_mass(QR_final)
              if not math.isnan(c1) and not math.isnan(c2):
              
                PILimage = Img.fromarray(QR_final)
                PILimage.save(os.path.join(dvejetainiuNuotraukuVieta,failoPavadinimas))

              else:
                print("Failed to save nuotrauka:" + failoPavadinimas + "\n try to make new object nuotrauka, with better background")
    
            else:
              print("Image reading failed")


def CDF(xt,yt,gx,gy):
  return math.sqrt((xt-gx)**2 + (yt-gy)**2)


def CDFSarasas(arr, gx,gy,perimetrasCTransponuotas, m = 64):
  n = len(arr) -1
  i = 0.0 
  aukst = float(n) / float(m) 
  return_arr = []
  skaitliukas = 0

  
  if arSektiIrasus > 0:
    plt.plot(perimetrasCTransponuotas[0,:], perimetrasCTransponuotas[1,:], 'k.')
    
  while i <= n:
    if skaitliukas < m:
      i += aukst
      skaitliukas += 1
      ii = math.floor(i)
      return_arr.append(CDF(arr[ii][0],arr[ii][1],gx,gy))
      if arSektiIrasus > 0:
        plt.plot([arr[ii][0],gx],[arr[ii][1],gy], '-og');
    else: 
      break

  if arSektiIrasus > 0:
    plt.axis('equal')
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.show()
    plt.plot(return_arr)

    plt.title("CDF - Centroidų atstumų funkcijos atstumai")
    plt.xlabel("Iteracija")
    plt.ylabel("Atstumas")
    plt.show()
  
  return return_arr


def imageDimentions(nuotr, path, process):
  # Įvairios formų analizės ir matavimai 
  # http://www.cyto.purdue.edu/cdroms/micro2/content/education/wirth10.pdf

  # Mases centras kuriuo apskaiciuojama zymiai greiciau negu   # print(centerofmass(nuotrauka))
  c1, c2 = ndimage.measurements.center_of_mass(nuotr) 

  # apskaičiavimas nuo centro koks yra aukstis y ir ilgis x
  xIlgis,y_ilgis = centro_WH(c1,c2,nuotr)

  # Gintaro ploto nuotraukoje apskaiciavimas :
  area_pixels, plotoSantykis = gintaroPlotas(nuotr)

  # gintaro aukštis ir plotis (aukstis and plotis). naudojantis cv2 irankiu
  x,y,w_bounds,h_bounds = cv2.boundingRect(nuotr) # Straight Bounding Rectangle

  # GINTARO objekto perimetras (object perimetras)
  perimetras, perimetroKoordinates = rastiPerimetra(nuotr, nuotr.shape[0], nuotr.shape[1]) # tikslesnis budas
  
  # gintaro diametras, tolimiausia pagrindine gintaro asis # bandymas apskaiciuoti 45laipsniu skersmens ilgi; DIAMETRAS
  perimetrasC,perimetrasCTransponuotas = gintaroMazDidzAsys(perimetroKoordinates)

  # 45 degree koordinates
  lenOf45Deg = angleBetweenTwoPoints(perimetrasC,c1,c2)

  # gintaro diametras, tolimiausia pagrindine gintaro asis
  # major_ax_d, major_ax  = major_axis(perimetrasC,perimetrasCTransponuotas)
  major_ax_d, major_ax  = pagindineAsis(perimetrasC,perimetrasCTransponuotas,c1, c2, w_bounds,h_bounds  )

  # major axis kampas degrees:
  major_ax_deg          = pagrindinesAsiesKampas(major_ax)

  # Kampų radimas, convex shape kapų kiekis, convex perimetras
  temp = np.array(perimetroKoordinates)
  if sum(temp[:,1]) >0 and sum(temp[:,0]) >= 0:
    convex_xy, convex_angles, convex_perimeter   = apgaubtasPerimetas(perimetroKoordinates) # maziausias apgaubtas plotas kuris uzdaro visa forma
    convexHull_area_size = apgaubtoHullPlotas(convex_xy)
    if arSektiIrasus:
      a1 = list(convex_xy[:,0])
      a2 = list(convex_xy[:,1])
      at = np.transpose([a1,a2])
      plt.plot(a1,a2, '-or');
      plt.axis('equal')
      plt.xticks(rotation=-90)
      plt.yticks(rotation=-90)
      plt.xlabel("Y")
      plt.ylabel("X")

      plt.show()
  else:
    convexHull_area_size = 0
    convex_xy = []
    convex_angles = 0
    convex_perimeter = 0

  #  isoperimetric quotient: Kompaktiškumo radimas, kai apvaliausias objektas yra kompaktišiausias 
  Q_compactness                   = rastiKompaktiskuma(area_pixels,perimetras)

  # Circularity or Roundness naudojantic covnex perimetru arba perimetru
  Q_roundness = rastiKompaktiskuma(area_pixels,convex_perimeter)

  
  # Boundary Descriptors arba Chain code arba Kraštinių nusakymas
  perimetrasC[:,0],perimetrasC[:,1] = rusiuotiXYPagalLaikrodzioRod(perimetrasC[:,0],perimetrasC[:,1])

  amber_chainLst = (grandinelesSarasas(perimetrasC))
  amber_chainLst = ''.join(str(e) for e in amber_chainLst)

  # Convexity
  convexity_q = ismatuotiConvexity(convex_perimeter, len(perimetroKoordinates))

  # CDF Centroid distance function
  CDF_array = CDFSarasas(perimetrasC,c1,c2,perimetrasCTransponuotas)

  # print(perimetrasC.shape)
  if arSektiIrasus:
    print("center of mass:",c1,c2)
    print("Image shape", nuotr.shape)
    print("PERIMETRAS",perimetras, "PERIMETRO KOORDINATES ILGIS", len(perimetroKoordinates))
    print("Gintaro diametras:", major_ax_d, "    Gintaro diametro X1Y1 X2Y2:",major_ax)
    print(str(major_ax_deg) + " kampas degrees")
    print("45 laipsniu kampo atstumas tarp krastiniu.", lenOf45Deg)
    print("Uždari convex array kampai",         convex_angles)
    print("Uždaro convex array perimetras ",    convex_perimeter)
    print("apgaubtoHullPlotas",                    convexHull_area_size)
    print("Isoperimetric quotient - compactness",  Q_compactness)
    print("Circularity or Roundness",           Q_roundness)
    print("Amber Chain List",                   amber_chainLst, "/n\nIt's lenght", len(amber_chainLst))
    print("CDF - Centroid distance function",   CDF_array)

  return xIlgis,y_ilgis, w_bounds,h_bounds,c1,c2 , perimetras, convex_angles, convex_perimeter, area_pixels, plotoSantykis,major_ax_d, major_ax, major_ax_deg , convexHull_area_size, Q_compactness, Q_roundness, amber_chainLst, CDF_array, lenOf45Deg

def main_data_preperation(sarasas, aplankalas, process, pasirinkimas = 0):
  df = pd.DataFrame(); df_main = pd.DataFrame()
  previous = 0; lastrow_amber = -1; imageCnt = 0;
  amber = []; centroidHist = []; perimeter_lst = []; center_distances = []; area_ratio_lst = []; pixels_inAshape = []; convexHull_area_size_list = []; convex_perimeterList = []; amber_majorAxisLength = []; amber_majorAxisLength_XY = []
  amber_ax_deg = []; Q_compactness_lst = []; Q_roundness_lst = []; convex_allAngles = [];
  amber_chainCode = list()

  for failoPavadinimas in sarasas:
      failoPatikra1 = pathlib.Path(os.path.join(aplankalas,failoPavadinimas))
      
      if failoPatikra1.exists():
        amber_itteration = int(failoPavadinimas.split(".")[0].split("-")[0])
        
        if lastrow_amber != amber_itteration:
          del amber_chainCode[:]
          del amber[:], centroidHist[:],perimeter_lst[:], center_distances[:], area_ratio_lst[:], pixels_inAshape[:], convexHull_area_size_list[:], convex_perimeterList[:], amber_majorAxisLength[:], amber_majorAxisLength_XY[:]
          del amber_ax_deg[:], Q_compactness_lst[:], Q_roundness_lst[:], convex_allAngles[:];
          imageCnt = 0 
        
        imageCnt+=1

        if imageCnt <= 3:
          nuotrauka = cv2.imread(aplankalas+ "/" + failoPavadinimas, 0)
          if nuotrauka is not None:
            print("Filename:",failoPavadinimas)
            nuotrauka = nezinomuReiksmiuPasalinimas(nuotrauka)

            """
            # Nuotraukos apvertimas vertikalia - Y - ašimi
            # nuotrauka = vertikalusApvertimas(nuotrauka)
            # nuotrauka = cv2.bitwise_not(nuotrauka)  # nuotraukos invertavimas
            # nuotrauka = cv2.resize(nuotrauka, (50, 50))        # pakeiciamas visu nuotrauku didis i standartini kad visu butu vienodas, reikalinga lyginimui pagal ploti gyli forma.
            """
          
            # Ivairus budai apskaiciuoti gintaro ilgi, auksti per centra ir kitus parametrus. 
            atsx,atsy,atsW,atsH, c1,c2, perimetras, convex_kampai, convex_perimetras, pikseliai, plotoSantykis, major_ax_d, major_ax_xy, major_ax_deg, convexHull_area_size, Q_compactness,Q_roundness, amber_chainLst, CDF_list, lenOf45Deg  = imageDimentions(nuotrauka,os.path.join(aplankalas,failoPavadinimas),process)
            
            data = { 
              'Filename': [failoPavadinimas],
              'amber_cnt': [failoPavadinimas.split(".")[0].split("-")[0]],
              'amber_picture': [failoPavadinimas.split(".")[0].split("-")[1]],
              'Xc': [c1], 
              'Yc': [c2],
              'plotoSantykis':[plotoSantykis],
              'plotas': [pikseliai],
              'plotis':[atsW],
              'aukstis':[atsH],
              'X_heigth':[atsx],
              'Y_heigth':[atsy],
              'diameter_len':[lenOf45Deg],
              'convex_area ':[convexHull_area_size],
              'convex_angles_cnt ':[convex_kampai],
              'convex_perimeter':[convex_perimetras],
              'amber_perimeter':[perimetras],
              'major_axis_len':[major_ax_d],
              'pagrindinesAsiesKampas':[major_ax_deg],
              'q_compactness':[Q_compactness],
              'q_roundness':[Q_roundness],
              'amber_chain':[amber_chainLst],
              'CDF':[CDF_list],
              'major_axis_coordinates':[major_ax_xy],
            }

            #
            df = df.append(pd.DataFrame(data) ,ignore_index=True)
            #
            
            centroidHist.append(c1);
            centroidHist.append(c2); 
            area_ratio_lst.append(plotoSantykis); 
            pixels_inAshape.append(pikseliai)
            amber.append(atsW); amber.append(atsH); 
            center_distances.append(atsx); center_distances.append(atsy);
            convexHull_area_size_list.append(convexHull_area_size)
            convex_allAngles.append(convex_kampai)
            convex_perimeterList.append(convex_perimetras)
            perimeter_lst.append(perimetras)
            amber_majorAxisLength.append(major_ax_d)
            amber_majorAxisLength_XY.append(major_ax_xy)
            amber_ax_deg.append(major_ax_deg)
            Q_compactness_lst.append(Q_compactness)
            Q_roundness_lst.append(Q_roundness)


            # Kas 3-is gintaro nuotraukas sutvarkome duomenis juos prideramai suporuojant
            if failoPavadinimas.split("-")[0] == previous and imageCnt == 3: 

              comboxXYZ, details, centroidsXYZ = kombinacijosSeka(amber, centroidHist,0)
              comboxXYZ_c, details_c = kombinacijosSeka(center_distances,[],0)

              if pasirinkimas == 0:
                data_main = { 'amber_cnt': [failoPavadinimas.split(".")[0].split("-")[0]],
                'Xh': [comboxXYZ[0]],
                'Xw': [comboxXYZ[1]],
                'Xz': [comboxXYZ[2]],
                # 'Gintaro WHD': [comboxXYZ],
                'Xc' : [centroidsXYZ[0]], 
                'Yc' : [centroidsXYZ[1]], 
                'Zc' : [centroidsXYZ[2]], 
                'diameter_len1':[(df['diameter_len'].iloc[-1])],
                'diameter_len2':[(df['diameter_len'].iloc[-2])],
                'diameter_len3':[(df['diameter_len'].iloc[-3])],
                # 'Centrdoid_XYZ_coordinates' : [centroidsXYZ], 
                # 'Amber_H_W_D_fromItsCenter':[comboxXYZ_c],
                'Xch': [comboxXYZ_c[0]],
                'Xcw':  [comboxXYZ_c[1]],
                'Xcz':   [comboxXYZ_c[2]],
                # 'Gintaro Plotas':[pixels_inAshape],
                'amber_area1':[pixels_inAshape[0]],
                'amber_area2':[pixels_inAshape[1]],
                'amber_area3':[pixels_inAshape[2]],
                'amber_area_sum':[sum(pixels_inAshape)],
                'amber_area_avg':[sum(pixels_inAshape)/len(pixels_inAshape)],
                'area_ratio1':[area_ratio_lst[0]],
                'area_ratio2':[area_ratio_lst[1]],
                'area_ratio3':[area_ratio_lst[2]],
                'area_ratio_sum':[sum(map(float,area_ratio_lst))],
                'area_ratio_avg':[sum(area_ratio_lst)/len(area_ratio_lst)],
                'convex_area1':[convexHull_area_size_list[0]],
                'convex_area2':[convexHull_area_size_list[1]],
                'convex_area3':[convexHull_area_size_list[2]],
                'convex_area_sum':[sum(convexHull_area_size_list)],
                'convex_area_sum':[sum(convexHull_area_size_list)/len(convexHull_area_size_list)],
                # 'ConvexHull gintaro kampai':[convex_allAngles],
                'convex_angles1':[convex_allAngles[0]],
                'convex_angles2':[convex_allAngles[1]],
                'convex_angles3':[convex_allAngles[2]],
                'convex_angles_sum':[sum(convex_allAngles)],
                'convex_angles_avg':[sum(convex_allAngles)/len(convex_allAngles)],
                # 'ConvexHull Perimetras':[convex_perimeterList],
                'convex_perimeter1':[convex_perimeterList[0]],
                'convex_perimeter2':[convex_perimeterList[1]],
                'convex_perimeter3':[convex_perimeterList[2]],
                'convex_perimeter_sum':[sum(map(float,convex_perimeterList))],
                'convex_perimeter_avg':[sum(convex_perimeterList)/len(convex_perimeterList)],
                # 'Gintaro Perimetras':[perimeter_lst],
                'amber_perimeter1':[perimeter_lst[0]],
                'amber_perimeter2':[perimeter_lst[1]],
                'amber_perimeter3':[perimeter_lst[2]],
                'amber_perimeter_sum':[sum(perimeter_lst)],
                'amber_perimeter_avg':[sum(perimeter_lst)/len(perimeter_lst)],
                # 'Gintaro diametras':[amber_majorAxisLength],
                'major_ax1':[amber_majorAxisLength[0]],
                'major_ax2':[amber_majorAxisLength[1]],
                'major_ax3':[amber_majorAxisLength[2]],
                'major_ax_sum':[sum(amber_majorAxisLength)],
                'major_ax_avg':[sum(amber_majorAxisLength)/len(amber_majorAxisLength)],
                # 'Gintaro diametro kampas':[amber_ax_deg],
                'major_ax_angle1':[amber_ax_deg[0]],
                'major_ax_angle2':[amber_ax_deg[1]],
                'major_ax_angle3':[amber_ax_deg[2]],
                'major_ax_angle_sum':[sum(amber_ax_deg)],
                'major_ax_angle_avg':[sum(amber_ax_deg)/len(amber_ax_deg)],
                # 'Gintaro kompaktiskumas':[Q_compactness_lst],
                'q_compactness1':[Q_compactness_lst[0]],
                'q_compactness2':[Q_compactness_lst[1]],
                'q_compactness3':[Q_compactness_lst[2]],
                'q_compactness_sum':[sum(Q_compactness_lst)],
                'q_compactness_avg':[sum(Q_compactness_lst)/len(Q_compactness_lst)],
                # 'Gintaro Apvalumas':[Q_roundness_lst],
                'q_roundness1':[Q_roundness_lst[0]],
                'q_roundness2':[Q_roundness_lst[1]],
                'q_roundness3':[Q_roundness_lst[2]],
                'q_roundness_sum':[sum(Q_roundness_lst)],
                'q_roundness_avg':[sum(Q_roundness_lst)/len(Q_roundness_lst)],
                'chaincode1':[(df['amber_chain'].iloc[-1])],
                'chaincode2':[(df['amber_chain'].iloc[-2])],
                'chaincode3':[(df['amber_chain'].iloc[-3])],
                'CDF1':[(df['CDF'].iloc[-1])],
                'CDF2':[(df['CDF'].iloc[-2])],
                'CDF3':[(df['CDF'].iloc[-3])],

                }
              elif pasirinkimas == 1:
                data_main = { 'amber_cnt': [failoPavadinimas.split(".")[0].split("-")[0]],
                'Xh': [comboxXYZ[0]],
                'Xw': [comboxXYZ[1]],
                'Xz': [comboxXYZ[2]],
                'Xc' : [centroidsXYZ[0]], 
                'Yc' : [centroidsXYZ[1]], 
                'Zc' : [centroidsXYZ[2]], 
                'amber_area_sum':[sum(pixels_inAshape)],
                'area_ratio_sum':[sum(map(float,area_ratio_lst))],
                'convex_area_sum':[sum(convexHull_area_size_list)],
                'convex_angles_sum':[sum(convex_allAngles)],
                'convex_perimeter_sum':[sum(map(float,convex_perimeterList))],
                'amber_perimeter_sum':[sum(perimeter_lst)],
                'major_ax_sum':[sum(amber_majorAxisLength)],
                'Gintaro diametro kampas su ojb SUM':[sum(amber_ax_deg)],
                'q_compactness_sum':[sum(Q_compactness_lst)],
                'q_roundness_sum':[sum(Q_roundness_lst)],
                'Chaincode':[amber_chainCode],
                }
              elif pasirinkimas == 2:
                data_main = { 'amber_cnt': [failoPavadinimas.split(".")[0].split("-")[0]],
                'Xh': [comboxXYZ[0]],
                'Xw': [comboxXYZ[1]],
                'Xz': [comboxXYZ[2]],
                'Xc' : [centroidsXYZ[0]], 
                'Yc' : [centroidsXYZ[1]], 
                'Zc' : [centroidsXYZ[2]], 
                'Xch': [comboxXYZ_c[0]],
                'Xcw':  [comboxXYZ_c[1]],
                'Xcz':   [comboxXYZ_c[2]],
                'amber_area_avg':[sum(pixels_inAshape)/len(pixels_inAshape)],
                'area_ratio_avg':[sum(area_ratio_lst)/len(area_ratio_lst)],
                'ConvexHull * gintaro plotas AVG':[sum(convexHull_area_size_list)/len(convexHull_area_size_list)],
                'convex_angles_avg':[sum(convex_allAngles)/len(convex_allAngles)],
                'convex_perimeter_avg':[sum(convex_perimeterList)/len(convex_perimeterList)],
                'amber_perimeter_avg':[sum(perimeter_lst)/len(perimeter_lst)],
                'major_ax_avg':[sum(amber_majorAxisLength)/len(amber_majorAxisLength)],
                'major_ax_angle_avg':[sum(amber_ax_deg)/len(amber_ax_deg)],
                'q_compactness_avg':[sum(Q_compactness_lst)/len(Q_compactness_lst)],
                'q_roundness_avg':[sum(Q_roundness_lst)/len(Q_roundness_lst)],
                'Chaincode':[amber_chainCode],
                }
              elif pasirinkimas == 3:
                data_main = { 'amber_cnt': [failoPavadinimas.split(".")[0].split("-")[0]],
                'Xh': [comboxXYZ[0]],
                'Xw': [comboxXYZ[1]],
                'Xz': [comboxXYZ[2]],
                'Xc' : [centroidsXYZ[0]], 
                'Yc' : [centroidsXYZ[1]], 
                'Zc' : [centroidsXYZ[2]], 
                'Xch': [comboxXYZ_c[0]],
                'Xcw':  [comboxXYZ_c[1]],
                'Xcz':   [comboxXYZ_c[2]],
                'amber_area1':[pixels_inAshape[0]],
                'amber_area2':[pixels_inAshape[1]],
                'amber_area3':[pixels_inAshape[2]],
                'area_ratio1':[area_ratio_lst[0]],
                'area_ratio2':[area_ratio_lst[1]],
                'area_ratio3':[area_ratio_lst[2]],
                'convex_area1':[convexHull_area_size_list[0]],
                'convex_area2':[convexHull_area_size_list[1]],
                'convex_area3':[convexHull_area_size_list[2]],
                'convex_angles1':[convex_allAngles[0]],
                'convex_angles2':[convex_allAngles[1]],
                'convex_angles3':[convex_allAngles[2]],
                'convex_perimeter1':[convex_perimeterList[0]],
                'convex_perimeter2':[convex_perimeterList[1]],
                'convex_perimeter3':[convex_perimeterList[2]],
                'amber_perimeter1':[perimeter_lst[0]],
                'amber_perimeter2':[perimeter_lst[1]],
                'amber_perimeter3':[perimeter_lst[2]],
                'amber_perimeter_sum':[sum(perimeter_lst)],
                'amber_perimeter_avg':[sum(perimeter_lst)/len(perimeter_lst)],
                'major_ax1':[amber_majorAxisLength[0]],
                'major_ax2':[amber_majorAxisLength[1]],
                'major_ax3':[amber_majorAxisLength[2]],
                'major_ax_angle1':[amber_ax_deg[0]],
                'major_ax_angle2':[amber_ax_deg[1]],
                'major_ax_angle3':[amber_ax_deg[2]],
                'q_compactness1':[Q_compactness_lst[0]],
                'q_compactness2':[Q_compactness_lst[1]],
                'q_compactness3':[Q_compactness_lst[2]],
                'q_roundness1':[Q_roundness_lst[0]],
                'q_roundness2':[Q_roundness_lst[1]],
                'q_roundness3':[Q_roundness_lst[2]],
                'Chaincode1':[(amber_chainCode[0])],
                'Chaincode2':[(amber_chainCode[1])],
                'Chaincode3':[(amber_chainCode[2])],
                }
              elif pasirinkimas == 4:
                data_main = { 'amber_cnt': [failoPavadinimas.split(".")[0].split("-")[0]],
                'XhPG': [comboxXYZ],
                'XcYZ' : [centroidsXYZ],  
                'XchPG': [comboxXYZ_c],
                'amber_area1':[pixels_inAshape],
                'area_ratio1':[area_ratio_lst],
                'convex_area1':[convexHull_area_size_list],
                'convex_angles1':[convex_allAngles],
                'convex_perimeter1':[convex_perimeterList],
                'amber_perimeter1':[perimeter_lst],
                'major_ax1':[amber_majorAxisLength],
                'major_ax_angle1':[amber_ax_deg],
                'q_compactness1':[Q_compactness_lst],
                'q_roundness1':[Q_roundness_lst],
                'Chaincode':[amber_chainCode],
                }
              
              df_main = df_main.append(pd.DataFrame(data_main) ,ignore_index=True)
            
            previous = failoPavadinimas.split("-")[0]
            lastrow_amber = int(failoPavadinimas.split(".")[0].split("-")[0])
          
            if arSektiIrasus > 0:
              cv2_imshow(nuotrauka)

  del amber_chainCode[:]

  del amber[:], centroidHist[:],perimeter_lst[:], center_distances[:], area_ratio_lst[:], pixels_inAshape[:], convexHull_area_size_list[:], convex_perimeterList[:], amber_majorAxisLength[:], amber_majorAxisLength_XY[:]
  del amber_ax_deg[:], Q_compactness_lst[:], Q_roundness_lst[:], convex_allAngles[:];
          
  return df, df_main

def angleBetweenTwoPoints(perimetrasC,c1,c2):
  m = 45      # https://en.wikipedia.org/wiki/Atan2 
  m1 = 135   
  m2 = 0     
  m3 = 180
  m4 = -90

  last_best_diffq1 = 999;
  last_best_diffq2 = 999;

  last_best_diffq3 = 999;
  last_best_diffq4 = 999;
  
  last_best_diffq5 = 999;
  last_best_diffq6 = 999;
  bestq1 = [0,0,0]
  bestq2 = [0,0,0]
  bestq3 = [0,0,0]
  bestq4 = [0,0,0]
  bestq5 = [0,0,0]

  for krastiniuKoordinates in perimetrasC:
    turimiRadianai = math.atan2(c2-krastiniuKoordinates[1], c1-krastiniuKoordinates[0])
    mydegrees = math.degrees(turimiRadianai)
    deg_ = abs((mydegrees) - m)
    deg_1 = abs((mydegrees) + m1)
    deg_2 = abs((mydegrees) - m2)
    deg_3 = abs((mydegrees) - m3)
    deg_4 = abs((mydegrees) + m4)
    deg_5 = abs((mydegrees) - m4)

    abs_degr  = (mydegrees)
    if deg_ < last_best_diffq1 and abs_degr > 0 and abs_degr < 90:
      last_best_diffq1 = deg_
      bestq1=[krastiniuKoordinates[0],krastiniuKoordinates[1],mydegrees]

    if deg_1 < last_best_diffq2 and abs_degr < -90 and abs_degr > -180:
      last_best_diffq2 = deg_1
      bestq2=[krastiniuKoordinates[0],krastiniuKoordinates[1],mydegrees]

    if deg_2 < last_best_diffq3:
      last_best_diffq3 = deg_2
      bestq3=[krastiniuKoordinates[0],krastiniuKoordinates[1],mydegrees]

    if deg_3 < last_best_diffq4:
      last_best_diffq4 = deg_3

      bestq4=[krastiniuKoordinates[0],krastiniuKoordinates[1],mydegrees]

    if deg_4 < last_best_diffq5:
      last_best_diffq5 = deg_4
      bestq5=[krastiniuKoordinates[0],krastiniuKoordinates[1],mydegrees]

    if deg_5 < last_best_diffq6:
      last_best_diffq6 = deg_5
      bestq6=[krastiniuKoordinates[0],krastiniuKoordinates[1],mydegrees]
  
  if arSektiIrasus:
    show_amber_45degaxis(bestq1,bestq2,bestq3,bestq4,c1,c2 ,bestq5,bestq6)
    print("Best 45 deg: ",bestq1, bestq2, bestq3,bestq4," DISTANCE BETWEEN THEM", CDF(bestq1[0],bestq1[1],bestq2[0],bestq2[1]) )
    print("Best Y: ", bestq3,bestq4," DISTANCE BETWEEN THEM", CDF(bestq3[0],bestq3[1],bestq4[0],bestq4[1]) )
    print("Best X: ", bestq5,bestq6," DISTANCE BETWEEN THEM", CDF(bestq6[0],bestq6[1],bestq5[0],bestq5[1]) )
  
  ret_answ = CDF(bestq1[0],bestq1[1],bestq2[0],bestq2[1])

  return ret_answ

def CDF(xt,yt,gx,gy):
  return math.sqrt((xt-gx)**2 + (yt-gy)**2)

def show_amber_45degaxis(best,bestq2,bestq3,bestq4,c1,c2,bestq5,bestq6):

  plt.plot((best[0],bestq2[0]),(best[1],bestq2[1]), '-og')

  plt.plot((bestq3[0],bestq4[0]),(bestq3[1],bestq4[1]), '-ob')
  
  plt.plot((bestq5[0],bestq6[0]),(bestq5[1],bestq6[1]), '-or')
  plt.scatter(c1,c2, marker='x',linewidth=2)

def resize_aspect_fit(resize_ratio, path, save_path , extt = 'PNG'):
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    if os.path.exists(save_path):
      if os.path.exists(path):
        dirs = os.listdir(path)
        
        for item in dirs:
          
          failoPatikra1 = pathlib.Path(os.path.join(save_path,item))
          if not failoPatikra1.exists():
            if item == '.jpg'  or item == '.jpeg' :
              extt = 'JPEG'
            elif item == '.png':
              extt = 'PNG'

            
            # print(path+item)
            if os.path.isfile((os.path.join(path,item))) and not failoPatikra1.exists():
              # print(failoPatikra1)
              # print(item)
              
              nuotrauka = Img.open((os.path.join(path,item)))
              file_path, extension = os.path.splitext((os.path.join(save_path,item)))
              new_image_height = int(nuotrauka.size[0] / (1/resize_ratio))
              new_image_length = int(nuotrauka.size[1] / (1/resize_ratio))
              nuotrauka = nuotrauka.resize((new_image_height,new_image_length), Img.ANTIALIAS)
              nuotrauka.save(file_path  + extension, extt, quality=90)
              print(file_path+extension)
      else:
        print("RAW nuotrauka location path doesn't exist")
    else:
      print("Save path doesn't exist")

###################################################################

csv_save_loc_group_orig = "/content/gdrive/My Drive/Bakalauras/Data/CSV/Ambers_group.csv"
csv_save_loc_one_orig = "/content/gdrive/My Drive/Bakalauras/Data/CSV/Ambers_one.csv"
smaller_sized_image_loc_orig = "/content/gdrive/My Drive/Bakalauras/Data/Resized_ambers"
binary_image_save_location_orig = "/content/gdrive/My Drive/Bakalauras/Data/BinaryImages"
raw_image_loc_orig = "/content/gdrive/My Drive/Bakalauras/Data/149ambers"
sorted_image_loc_orig = "/content/gdrive/My Drive/Bakalauras/Data/cluster/"


csv_save_loc_group = "/content/gdrive/My Drive/Bakalauras/Data/test/CSV/Ambers_group.csv"
csv_save_loc_csv = "/content/gdrive/My Drive/Bakalauras/Data/test/CSV/Ambers.csv"
csv_save_loc_one = "/content/gdrive/My Drive/Bakalauras/Data/test/CSV/Ambers_one.csv"
smaller_sized_image_loc = "/content/gdrive/My Drive/Bakalauras/Data/test/ResizedImages"
binary_image_save_location = "/content/gdrive/My Drive/Bakalauras/Data/test/BinaryImages"
raw_image_loc = "/content/gdrive/My Drive/Bakalauras/Data/test/RawAmberPictures"
sorted_image_loc = "/content/gdrive/My Drive/Bakalauras/Data/test/cluster/"

kmean_model_loc = "/content/gdrive/My Drive/Bakalauras/Data/K-Mean-MODEL"
csv_rename_loc = "/content/gdrive/My Drive/Bakalauras/Data/test/ResizedImages"

dirfmtT = "%4d-%02d-%02d %02d:%02d:%02d"
dirfmt = "/content/gdrive/My Drive/Bakalauras/Data/test/cluster/%4d-%02d-%02d %02d:%02d:%02d"


def operation_1(aplankalas, dvejetainiuNuotraukuVieta, log_chois = 0):
  ##### Operacijos ir su seka vykdomasis kodas
  global arSektiIrasus
  arSektiIrasus = log_chois

  # I-a operacija 
  start = time.time()

  #aplankalas = "/content/gdrive/My Drive/Bakalauras/Data/149ambers"
    
  nuskaitytiOriginaliaNuotrauka_KonvertuotiIOriginaliaApkarpyta(aplankalas,dvejetainiuNuotraukuVieta)
  end = time.time()
  print(end - start, "s")

# II-a operacija
def operation2(folder_loacation, csv_location, log_chois = 0):
    testi = True
    global arSektiIrasus
    arSektiIrasus = log_chois

    try:
      folder_lst = os.listdir(folder_loacation); 
    except FileNotFoundError:
          print("Wrong binary nuotrauka location")
    else:

      try:
        p1 = csv_location.split(".")[0]
        p2 = csv_location.split(".")[1]
      except IndexError:
        print("Wrong csv failoPavadinimas")
      else:
        if not os.path.exists(os.path.dirname(csv_location)):
          try:
            os.makedirs(os.path.dirname(csv_location))
          except FileNotFoundError:
            testi = False
        if testi:
          sarasas = sorted( folder_lst, key=lambda a: int(a.split(".")[0].split("-")[0]),reverse=False);

          process = psutil.Process(os.getpid())
          print(dydzioFormatas(process.memory_info().rss))

          start = time.time()

          df_F, df_main_F = main_data_preperation(folder_lst, folder_loacation, process, 0)

          print(dydzioFormatas(process.memory_info().rss))   

          display(df_F)
          display(df_main_F)
          
          end = time.time()
          print(end - start, "s")
          operation_saveCsv(df_main_F , f"{p1}_group.{p2}")
          operation_saveCsv(df_F , f"{p1}_one.{p2}")
        else:
          print("Can't create csv directory for specified file, path is incorrect.")


def operation_saveCsv(df_main_F, csv_location = '/content/gdrive/My Drive/Bakalauras/Data/CSV/ambers_export.csv',):
    if not os.path.exists(os.path.dirname(csv_location)):
      try:
        os.makedirs(os.path.dirname(csv_location))
      except FileNotFoundError:
        print("Can't create directory, specified path is incorrect.")
      else:
        df_main_F.to_csv (csv_location, index = False, header=True)
    else: 
      df_main_F.to_csv (csv_location, index = False, header=True)

def operation6_datastructure_correlation(csv_folder):
  testi = True
  try:
    amber_csv_data = pd.read_csv(csv_folder) 
  except FileNotFoundError:
    print("Wrong CSV location")
    testi = False
  if testi:
    amber_csv_data.info()

    while True:
      try:
          t1 = str(input("Enter DataStruct end limiter ( or leave empty for default colums )\n "))
          if t1 == "" and len(amber_csv_data.columns) <= 23:
            t1 = str("3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19")
          elif t1 == "" and  len(amber_csv_data.columns) > 25:
                t1 = str("1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67")
      except ValueError:
          if t1 == "" and  len(amber_csv_data.columns) <= 23:
            t1 = str("3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19")
          elif t1 == "" and  len(amber_csv_data.columns) > 25:
                t1 = str("1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67")
          else:
            print("You entered not a number! Try again.")
            continue
      else:
          break 
    
    map_s = map(converter,t1.split())
    map_s = [x for x in map_s if x >= 0 and x <= len(amber_csv_data.columns)]
    s_copy = amber_csv_data.iloc[:, list(map_s)] 
    
    columnNames = s_copy.columns.values.tolist()

    for dropNames in [string for string in columnNames if 'amber_cnt' in string or 'chaincode' in string or 'amber_picture' in string or 'amber_chain' in string or 'CDF' in string or 'major_axis_coordinates' in string or 'Filename' in string or 'amber_cnt' in string]:
            s_copy = s_copy.drop([dropNames],axis = 1)

    print("Headers",list(amber_csv_data.columns.values),len(list(amber_csv_data.columns.values)))
    print("Headers",list(s_copy.columns.values))
    if len(list(s_copy.columns.values)) > 1:
      kaipKopija=((s_copy-s_copy.min())/(s_copy.max()-s_copy.min()))*20
      corrMatrix = s_copy.corr()
      fig, ax = plt.subplots(figsize=(40,40)) 
      sn.heatmap(corrMatrix, annot=True, linewidths=.5, ax=ax)
      plt.show()
      corrMatrix.to_csv("/content/gdrive/My Drive/Bakalauras/correlationCSV.csv")
    else:
      print(" !! Ammount of columns is too small, change column range !! ")


def renameByRepetition(path, repetition):
  if os.path.exists(path):
    dirs = os.listdir(path)
    dirs = natsort.natsorted(dirs,reverse=False)
    i = 1;
    j = 1;
    for item in dirs:
      old_file = pathlib.Path(os.path.join(path,item))
      if os.path.exists(old_file):
        fileType = item.split(".")[1]
        new_file = os.path.join(path, f"{i}-{j}.{fileType}")
        print(new_file, item)
        os.rename(old_file, new_file)
        if j >= repetition:
          j = 1
          i = i + 1 
        else:
          j = j + 1
  else:
    print("Path doesn't exist")
      
#! usr/bin/python

def converter(l):
    try:
        return ast.literal_eval(l)
    except ValueError:
        return l

def DFT_slow(x):
    """Compute the discrete Fourier Transform of the 1D array x"""
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    n = np.arange(N)
    k = n.reshape((N, 1))
    M = np.exp(-2j * np.pi * k * n / N)
    return np.dot(M, x)

def operation5_loadcsv_Kmeans(csv_folder, cluster_loc = sorted_image_loc ,image_loc = binary_image_save_location, relearn_query = 0,  model_loc_folder = kmean_model_loc, normalization = 0, move_images = 0):#smaller_sized_image_loc): binary_image_save_location          n_klasteriuDydis, t1,t2
    testi = True
    try:
      amber_csv_data = pd.read_csv(csv_folder) 
    except FileNotFoundError:
      testi = False
      print(f"   !!! Alert !!!\n   Wrong CSV location\n   !!! Alert !!!")

    
    if not os.path.exists(os.path.dirname(cluster_loc)):
      try:
        os.makedirs(os.path.dirname(cluster_loc))
      except FileNotFoundError:
        print("Wrong Clustered nuotrauka save location")
        testi = False
    
    if not os.path.exists(os.path.dirname(cluster_loc)):
      try:
        aplankaloSarasas = os.listdir(image_loc)
      except FileNotFoundError:
        print("Wrong original nuotrauka reading location")
        testi = False

    if testi:

      if relearn_query == 0:
        while True:
          try:
              n_klasteriuDydis=int(input("Enter N cluster size for Kmean\n "))
              if n_klasteriuDydis < 2 or n_klasteriuDydis >= len(amber_csv_data):
                print(f" cluster size should be: 2 <= n and n >= {len(amber_csv_data)}")
                continue
          except ValueError:
              print("You entered not a number! Try again.")
              continue
          else:
              break 
      elif relearn_query == 1:
        
        kVidurkiai =  pickle.load(open(f"{model_loc_folder}/kmean.pkl", "rb"))
        kmean_settings = pickle.load(open(f"{model_loc_folder}/kmean_settings.pkl", "rb"))
        t1 =  kmean_settings[0]
        n_klasteriuDydis = len(kVidurkiai.cluster_centers_)
        print(f" Last model has this amount of clusters {n_klasteriuDydis}, from this file: {model_loc_folder}/kmean.pkl")

      amber_csv_data.info()
      
      if relearn_query == 0:
        while True:
          try:
              t1 = str(input("Enter DataStruct end limiter ( or leave empty for default colums )\n "))
              if t1 == "" and len(amber_csv_data.columns) <= 23:
                t1 = str("3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19")
              elif t1 == "" and  len(amber_csv_data.columns) > 25:
                t1 = str("3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67")
          except ValueError:
              if t1 == "" and  len(amber_csv_data.columns) <= 23:
                t1 = str("3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19")
              elif t1 == "" and  len(amber_csv_data.columns) > 25:
                t1 = str("3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67")
              else:
                print("You entered not a number! Try again.")
                continue
          else:
              break 
      elif relearn_query == 1:
        print("Loaded DataStruct end limiter", t1)
      columnsInput = list(map(converter,t1.split()))

      if len(columnsInput) == 1 and amber_csv_data.columns.values.tolist()[columnsInput[0]] == 'CDF':
        arr = []
        
        kaipKopija = amber_csv_data.copy()
        s_copies = amber_csv_data["CDF"].to_numpy()
                
        for s_copy in s_copies:
          scp1 = np.asarray(json.loads(s_copy), dtype=float)
          fttOfCDF = np.fft.fft(json.loads(s_copy))
          n = np.arange(fttOfCDF.shape[0])

          arr.append(scp1.tolist())
        
        print(arr)
        arr = np.array(arr)
        v = arr   # 
        arr[:, :] = (v) / (v.sum())

        # Normalizacija

        if relearn_query == 0:
          kVidurkiai = KMeans(n_clusters = n_klasteriuDydis)
          yKvidurkiai = kVidurkiai.fit_predict(arr)
          pickle.dump(kVidurkiai, open(f"{model_loc_folder}/kmean.pkl", "wb"))
        elif relearn_query == 1:
          # try:
          yKvidurkiai = kVidurkiai.fit_predict(X)
          
        ##############################
        kaipKopija.insert(0, 'Cluster', yKvidurkiai)

        kaipKopija = (kaipKopija.sort_values(by=['Cluster']))
    
        aplankaloSarasas = os.listdir(image_loc)
        print_image_list = sorted( aplankaloSarasas, key=lambda a: int(a.split(".")[0].split("-")[0]),reverse=True )  
                   
        today = datetime.now()

        if today.hour < 12:
            aukst = "00"

        else:
            aukst = "12"

        now = time.localtime()[0:6]
        print(cluster_loc, now)
        save_path = os.path.join(cluster_loc, dirfmtT)
        save_path = save_path % now

        if not os.path.exists(os.path.dirname(cluster_loc)):
                os.makedirs(os.path.dirname(cluster_loc))
        os.mkdir(save_path)
        
        for i in range(n_klasteriuDydis):
          os.mkdir(os.path.join(save_path,str(i)+"cluster"))

        for index, row in kaipKopija.iterrows():

            if 'Filename' in row:
              print(row['Cluster'], row['Filename'])
              matching = str(row['Filename'])
              tmp_path = pathlib.Path(os.path.join(image_loc,matching))
              failoPatikra2 = pathlib.Path(tmp_path)
              if failoPatikra2.exists() == True :
                shutil.copy2(tmp_path, (os.path.join(save_path,str(row['Cluster'])+"cluster")) )

            if not 'Filename' in row and 'amber_cnt' in row:
              loc_str = str(row['amber_cnt'])+"-"
              matching = [s for s in print_image_list if s.startswith(loc_str)]
              print("Cluster: ", row['Cluster'], matching)
              for img_loc in matching:
                tmp_path = pathlib.Path(os.path.join(image_loc,img_loc))
                failoPatikra2 = pathlib.Path(tmp_path)
                if failoPatikra2.exists() == True :
                  shutil.copy2(tmp_path, (os.path.join(save_path,str(row['Cluster'])+"cluster")) )

        print("\n\nClustered data save path", save_path, "\n\n")

      else:

        start = time.time()
          
        map_s = map(converter,t1.split())
        map_s = [x for x in map_s if x >= 0 and x <= len(amber_csv_data.columns)]
        s_copy = amber_csv_data.iloc[:, list(map_s)]

        columnNames = s_copy.columns.values.tolist()

        for dropNames in [string for string in columnNames if 'amber_cnt' in string or 'chaincode' in string or 'amber_picture' in string or 'amber_chain' in string or 'CDF' in string or 'major_axis_coordinates' in string or 'Filename' in string or 'amber_cnt' in string]:
          s_copy = s_copy.drop([dropNames],axis = 1)
        
        if len(s_copy.columns) >= 1:
          s_copy = s_copy.infer_objects()

          kaipKopija = amber_csv_data.copy()
          
          # Normalizacija
          
          if relearn_query == 0:
            smin = [];
            smax = [];
            for column in s_copy.columns:
              smin.append(s_copy[column].min())
              smax.append(s_copy[column].max())
              s_copy[column] = (s_copy[column] -  s_copy[column].min() ) / (s_copy[column].max() -  s_copy[column].min() )  

          elif relearn_query == 1:
            smin = kmean_settings[2]
            smax = kmean_settings[1]
          
            i = 0
            for column in s_copy.columns:
              s_copy[column] = (s_copy[column] - smin[i]) / (smax[i] - smin[i])    
              i += 1
                      
          X = s_copy.iloc[:, :].values

          if len(X) == 0:
            print(f"   !!! Alert !!!\n   Wrong file or amount of values is too low \n   !!! Alert !!!")
          elif n_klasteriuDydis < len(X) or n_klasteriuDydis > 2:
            
            if relearn_query == 0:
              kVidurkiai = KMeans(n_clusters = n_klasteriuDydis)
              yKvidurkiai = kVidurkiai.fit_predict(X)
              pickle.dump(kVidurkiai, open(f"{model_loc_folder}/kmean.pkl", "wb"))
              pickle.dump([t1, smax, smin], open(f"{model_loc_folder}/kmean_settings.pkl", "wb"))
            elif relearn_query == 1:

              yKvidurkiai = kVidurkiai.fit_predict(X)
              
            end = time.time()
            print(end - start, "s")
            kaipKopija.insert(0, 'Cluster', yKvidurkiai)

            kaipKopija = (kaipKopija.sort_values(by=['Cluster']))
           
            aplankaloSarasas = os.listdir(image_loc)
            print_image_list = sorted( aplankaloSarasas, key=lambda a: int(a.split(".")[0].split("-")[0]),reverse=True )  
             
            today = datetime.now()

            if today.hour < 12:
                aukst = "00"

            else:
                aukst = "12"

            now = time.localtime()[0:6]
            print(cluster_loc, now)
            save_path = os.path.join(cluster_loc, dirfmtT)
            save_path = save_path % now

            if not os.path.exists(os.path.dirname(cluster_loc)):
              try:
                os.makedirs(os.path.dirname(cluster_loc))
              except FileNotFoundError:
                print("Wrong Clustered nuotrauka save location")
                testi = False

            if testi:
              os.mkdir(save_path)
              if move_images == 0: 
                for i in range(n_klasteriuDydis):
                  os.mkdir(os.path.join(save_path,str(i)+"cluster"))
                
                for index, row in kaipKopija.iterrows():

                    if 'Filename' in row:

                      print(row['Cluster'], row['Filename'])
                      matching = str(row['Filename'])
                      tmp_path = pathlib.Path(os.path.join(image_loc,matching))
                      failoPatikra2 = pathlib.Path(tmp_path)

                      if failoPatikra2.exists() == True :
                        shutil.copy2(tmp_path, (os.path.join(save_path,str(row['Cluster'])+"cluster")) )

                    if not 'Filename' in row and 'amber_cnt' in row:

                      loc_str = str(row['amber_cnt'])+"-"
                      matching = [s for s in print_image_list if s.startswith(loc_str)]
                      print("Cluster: ", row['Cluster'], matching)

                      for img_loc in matching:
                        tmp_path = pathlib.Path(os.path.join(image_loc,img_loc))
                        failoPatikra2 = pathlib.Path(tmp_path)

                        if failoPatikra2.exists() == True :
                          shutil.copy2(tmp_path, (os.path.join(save_path,str(row['Cluster'])+"cluster")) )

                print("\n\nClustered data save path", save_path, "\n\n")
            

            p1 = save_path + "/" +csv_folder.split("/")[-1].split(".")[0] + "_results.csv"
            print("\n\nClustered csv details saved in", p1 , "\n\n")
            operation_saveCsv(kaipKopija , f"{p1}")
            ############################################
            # Silhouette 

            print("\n\nThis Silhouette coefficient dimension tells, how points in one cluster are far away from another cluster;", "\nThe silhouette coefficient of whole cluster is the average of all clusters silhouette coefficients, the value can differ between [-1,1];")
            print(" - A value close to coefficient +1 indicates that the clusters are far from each other")
            print(" - A value close to coefficient  0 indicates that the clusters are close to each other and they might overlap")
            print(" - A value close to coefficient -1 indicates that certain points have been assigned incorrectly, which mignt be not for that cluster.\n")

            siluetoVidurkis = siluetoTaskai(X, yKvidurkiai)
            print("For n_clusters =", n_klasteriuDydis,
                  "The average silhouette score is: ", siluetoVidurkis," (marked in red dashed line)")

            # Apskaiciuojami sipuetai sio pavyzdzio
            pavyzdzioSiluetoReiksmes = silhouette_samples(X, yKvidurkiai)

            y_apatinis = 10
            for i in range(n_klasteriuDydis):

                ith_klasteriopavyzdzioSiluetoReiksmes = \
                pavyzdzioSiluetoReiksmes[yKvidurkiai == i]

                ith_klasteriopavyzdzioSiluetoReiksmes.sort()

                klasterioDydis_i = ith_klasteriopavyzdzioSiluetoReiksmes.shape[0]
                y_virsutinis = y_apatinis + klasterioDydis_i

                color = cm.nipy_spectral(float(i) / n_klasteriuDydis)
                plt.fill_betweenx(np.arange(y_apatinis, y_virsutinis),
                                  0, ith_klasteriopavyzdzioSiluetoReiksmes,
                                  facecolor=color, edgecolor=color, alpha=0.7)

                plt.text(-0.05, y_apatinis + 0.5 * klasterioDydis_i, str(i))

                y_apatinis = y_virsutinis + 10  

            plt.xlabel("The silhouette coefficient values")
            plt.ylabel("Cluster label")

            plt.axvline(x = siluetoVidurkis, color="red", linestyle="--")

            plt.yticks([])  
            plt.xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

            plt.suptitle(("Silhouette analysis of KMeans clustering "
                          "with n_clusters = %d" % n_klasteriuDydis),
                        fontsize=14, fontweight='bold')
            plt.show()
            ############################################
            # Alkunes metodo inercija
            if len(X) >= n_klasteriuDydis:
              distortions = []
              if len(X) >= n_klasteriuDydis + n_klasteriuDydis:
                K = range(1, (n_klasteriuDydis + n_klasteriuDydis ))
              else: 
                K = range(1, (n_klasteriuDydis ))
              print("\n\nInertia graph. Which shows how far away are clusters between each other;", "\nIt is suggested to avoid over-clustering and to choose the number of clusters that is close to the rise of the inertia curve;\n")
              distortions = []

              for k in K:
                  kVidurkiuModelis = KMeans(n_clusters = k)#, init = 'k-means++', random_state = 42) #(n_clusters=k) # (n_clusters = n_klasteriuDydis, init = 'k-means++', random_state = 42)
                  kVidurkiuModelis.fit(X)
                  distortions.append(kVidurkiuModelis.inertia_)

              plt.figure(figsize=(8,4))
              plt.plot(K, distortions, 'bx-')
              plt.xlabel('k - amount of clusters')
              plt.ylabel('Inertia')
              plt.title('The Elbow Method, which shows the optimal k')
              plt.show()
              
            else:
              print("Cant display Elbow method analysis, the amount of samples is too low")
            ############################################
          else:
            print(f"   !!! Alert !!!\n   You need to change N - cluster size [2,{len(X)}]\n   !!! Alert !!!")
        else:
          print(f"   !!! Alert !!!\n   You need to enter more avaible values\n {len(s_copy.columns)}  !!! Alert !!!")

def operation_SOM(csv_folder, n_klasteriuDydis, cluster_loc = sorted_image_loc ,image_loc = binary_image_save_location):
  arr = []
  amber_csv_data = pd.read_csv(csv_folder) 
  amber_csv_data = amber_csv_data.sort_values(by=['amber_cnt'])
  print(amber_csv_data.columns.values.tolist())
  print(amber_csv_data.columns.values.tolist()[21])
  print(list(amber_csv_data.columns.values.tolist()))
  print(list(amber_csv_data.columns.values.tolist())[21])
  if amber_csv_data.columns.values.tolist()[21] == 'CDF1':
    print("++++++++++++++++++++++")
  kaipKopija = amber_csv_data.copy()
  
  s_copies = amber_csv_data["CDF1"].to_numpy()
  i = 0
  for s_copy in s_copies:
    arr1 = json.loads(s_copy)

    scp1 = np.asarray(arr1, dtype=float)
    fttOfCDF = np.fft.fft(arr1)
    n = np.arange(fttOfCDF.shape[0])

    plt.plot(n, fttOfCDF, '-bo')
    plt.title("Furje Transformacija")
    plt.ylabel("Normalizuotų Furjė koeficientų dydis")
    plt.xlabel("Dažnis")
    plt.show()
    
    print(arr1)
    print(n)
    print(i)
    
    plt.plot(n, arr1)
    plt.title("CDF - Centroidų atstumų funkcijos atstumai")
    plt.xlabel("Iteracija")
    plt.ylabel("Atstumas")
    plt.show()


    i = i + 1
    arr.append(scp1.tolist())
  
  print(arr)
  arr = np.array(arr)
  v = arr   # 
  arr[:, :] = (v) / (v.sum())
    
  kVidurkiai = KMeans(n_clusters = n_klasteriuDydis)

  yKvidurkiai = kVidurkiai.fit_predict(arr)

  ##############################

  kaipKopija.insert(0, 'Cluster', yKvidurkiai)
  kaipKopija = (kaipKopija.sort_values(by=['Cluster']))

  aplankaloSarasas = os.listdir(image_loc)
  print_image_list = sorted( aplankaloSarasas, key=lambda a: int(a.split(".")[0].split("-")[0]),reverse=True )  
      
  today = datetime.now()

  if today.hour < 12:
      aukst = "00"
  else:
      aukst = "12"

  now = time.localtime()[0:6]
  print(cluster_loc, now)
  save_path = os.path.join(cluster_loc, dirfmtT)
  save_path = save_path % now
  if not os.path.exists(os.path.dirname(cluster_loc)):
          os.makedirs(os.path.dirname(cluster_loc))
  os.mkdir(save_path)
  
  for i in range(n_klasteriuDydis):
    os.mkdir(os.path.join(save_path,str(i)+"cluster"))

  for index, row in kaipKopija.iterrows():

      if 'Filename' in row:
        print(row['Cluster'], row['Filename'])
        matching = str(row['Filename'])
        tmp_path = pathlib.Path(os.path.join(image_loc,matching))
        failoPatikra2 = pathlib.Path(tmp_path)
        if failoPatikra2.exists() == True :
          shutil.copy2(tmp_path, (os.path.join(save_path,str(row['Cluster'])+"cluster")) )

      if not 'Filename' in row and 'amber_cnt' in row:
        loc_str = str(row['amber_cnt'])+"-"
        matching = [s for s in print_image_list if s.startswith(loc_str)]
        print("Cluster: ", row['Cluster'], matching)
        for img_loc in matching:
          tmp_path = pathlib.Path(os.path.join(image_loc,img_loc))
          failoPatikra2 = pathlib.Path(tmp_path)
          if failoPatikra2.exists() == True :
            shutil.copy2(tmp_path, (os.path.join(save_path,str(row['Cluster'])+"cluster")) )

  print("\n\nClustered data save path", save_path, "\n\n")

  
def operation6_loadcsv_SOM(csv_folder, cluster_loc = sorted_image_loc ,image_loc = binary_image_save_location, relearn_query = 0,  model_loc_folder = kmean_model_loc, normalization = 0, move_images = 0):#smaller_sized_image_loc): binary_image_save_location          n_klasteriuDydis, t1,t2
    testi = True
    try:
      amber_csv_data = pd.read_csv(csv_folder) 
    except FileNotFoundError:
      testi = False
      print(f"   !!! Alert !!!\n   Wrong CSV location\n   !!! Alert !!!")

    
    if not os.path.exists(os.path.dirname(cluster_loc)):
      try:
        os.makedirs(os.path.dirname(cluster_loc))
      except FileNotFoundError:
        print("Wrong Clustered nuotrauka save location")
        testi = False
    
    if not os.path.exists(os.path.dirname(cluster_loc)):
      try:
        aplankaloSarasas = os.listdir(image_loc)
      except FileNotFoundError:
        print("Wrong original nuotrauka reading location")
        testi = False

    if testi:
      if relearn_query == 0:
        while True:
          try:
              n_map_size=int(input("Enter N size for SOM\n "))
          except ValueError:
              print("You entered not a number! Try again.")
              continue
          else:
              break 
        while True:
          try:
              m_map_size=int(input("Enter M size for SOM\n "))
          except ValueError:
              print("You entered not a number! Try again.")
              continue
          else:
              break 
      elif relearn_query == 1:
      #  Permokinti algoritma per nauja su naujais duomenimis isemant pickle duomenis ir idedant i SOM

        som =  pickle.load(open(f"{model_loc_folder}/som.pkl", "rb"))
        som_settings = pickle.load(open(f"{model_loc_folder}/som_settings.pkl", "rb"))
        t1 =  som_settings[0]
        
        n_map_size = som_settings[3]
        m_map_size = som_settings[4]
        print(f" This SOM map model dimensions is {m_map_size}x{n_map_size}, from this file: {model_loc_folder}/som.pkl")

      amber_csv_data.info()
            
      if relearn_query == 0:
        while True:
          try:
              t1 = str(input("Enter DataStruct end limiter ( or leave empty for default colums )\n "))
              if t1 == "" and len(amber_csv_data.columns) <= 23:
                t1 = str("3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19")
              elif t1 == "" and  len(amber_csv_data.columns) > 25:
                t1 = str("3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67")
          except ValueError:
              if t1 == "" and  len(amber_csv_data.columns) <= 23:
                t1 = str("3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19")
              elif t1 == "" and  len(amber_csv_data.columns) > 25:
                t1 = str("3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67")
              else:
                print("You entered not a number! Try again.")
                continue
          else:
              break 
      elif relearn_query == 1:
        print("Loaded DataStruct end limiter", t1)
      start = time.time()
      columnsInput = list(map(converter,t1.split()))
      if len(columnsInput) > 1 and amber_csv_data.columns.values.tolist()[columnsInput[0]] != 'CDF':
              
        map_s = map(converter,t1.split())
        map_s = [x for x in map_s if x >= 0 and x <= len(amber_csv_data.columns)]
        s_copy = amber_csv_data.iloc[:, list(map_s)]#-5:-2] # df.infer_objects()
        columnNames = s_copy.columns.values.tolist()

        for dropNames in [string for string in columnNames if 'amber_cnt' in string or 'chaincode' in string or 'amber_picture' in string or 'amber_chain' in string or 'CDF' in string or 'major_axis_coordinates' in string or 'Filename' in string or 'amber_cnt' in string]:
          s_copy = s_copy.drop([dropNames],axis = 1)
        
        if len(s_copy.columns) >= 1:
          s_copy = s_copy.infer_objects()
          kaipKopija = amber_csv_data.copy()

          # min-max normalizavimas 
          # Normalizacija

          if relearn_query == 0:
            smin = [];
            smax = [];
            for column in s_copy.columns:
              smin.append(s_copy[column].min())
              smax.append(s_copy[column].max())
              s_copy[column] = (s_copy[column] -  s_copy[column].min() ) / (s_copy[column].max() -  s_copy[column].min() )  
          elif relearn_query == 1:
            smin = som_settings[2]
            smax = som_settings[1]
            i = 0
            for column in s_copy.columns:
              s_copy[column] = (s_copy[column] - smin[i]) / (smax[i] - smin[i])    
              i += 1
          
          X = s_copy.iloc[:, :].values

          if len(X) == 0:
            print(f"   !!! Alert !!!\n   Wrong file or amount of values is too low \n   !!! Alert !!!")

          elif n_map_size > 2 and  m_map_size > 1 and len(X) > 0:

            if relearn_query == 0:
              som = SOM(m=n_map_size , n=m_map_size, dimensija=len(s_copy.columns))
              '''
              m - tai vetrikalios pozicijos SOM žemėlapio dimensija;
              n - tai horizontalios padeties SOM žemėlapio dimensija;
              dimensija - tai iškirtinių savybių (stulpelių) kiekis;
              sigma = 1 - tai pasirinktinas parametras keičiantis regavimus į skirtingus duomenų svorius;
              lr = 1 - tai žingsnio dydis atnaujinant SOM svorius;
              max_iter = 3000 - tai parametras nusakantis po kiek iteracijų nustoti trenitavimą.
              '''
              som.fit(X)
              '''
              fit(X, epochs=1, shuffle=True)
              epochs - tai kiek kartų kartoti pradinius duomenis.
              shuffle - tai pasirinkimas ar sumaišyti duomenis ar ne prieš perduodant duomenis.


              '''
              y_som = som.predict(X)

              pickle.dump(som, open(f"{model_loc_folder}/som.pkl", "wb"))
              pickle.dump([t1, smax, smin, n_map_size, m_map_size ], open(f"{model_loc_folder}/som_settings.pkl", "wb"))
            elif relearn_query == 1:
              som.fit(X)
              y_som = som.predict(X)
              pickle.dump(som, open(f"{model_loc_folder}/som.pkl", "wb"))
          
            kaipKopija.insert(0, 'Cluster', y_som)
            kaipKopija = (kaipKopija.sort_values(by=['Cluster']))

            
            end = time.time()
            print(end - start, "s")

            aplankaloSarasas = os.listdir(image_loc)
            print_image_list = sorted( aplankaloSarasas, key=lambda a: int(a.split(".")[0].split("-")[0]),reverse=True )  
            
            today = datetime.now()

            if today.hour < 12:
                aukst = "00"
            else:
                aukst = "12"

            now = time.localtime()[0:6]
            print(cluster_loc, now)
            save_path = os.path.join(cluster_loc, dirfmtT)
            save_path = save_path % now

            if not os.path.exists(os.path.dirname(cluster_loc)):
              try:
                os.makedirs(os.path.dirname(cluster_loc))
              except FileNotFoundError:
                print("Wrong Clustered nuotrauka save location")
                testi = False

            if testi:
              os.mkdir(save_path)
              if move_images == 0: 
                for row in kaipKopija['Cluster'].unique():
                  os.mkdir(os.path.join(save_path,str(row)+"cluster"))
                
                for index, row in kaipKopija.iterrows():

                    if 'Filename' in row:
                      print(row['Cluster'], row['Filename'])
                      matching = str(row['Filename'])
                      tmp_path = pathlib.Path(os.path.join(image_loc,matching))
                      failoPatikra2 = pathlib.Path(tmp_path)
                      if failoPatikra2.exists() == True :
                        shutil.copy2(tmp_path, (os.path.join(save_path,str(row['Cluster'])+"cluster")) )

                    if not 'Filename' in row and 'amber_cnt' in row:
                      loc_str = str(row['amber_cnt'])+"-"
                      matching = [s for s in print_image_list if s.startswith(loc_str)]
                      print("Cluster: ", row['Cluster'], matching)
                      for img_loc in matching:
                        tmp_path = pathlib.Path(os.path.join(image_loc,img_loc))
                        failoPatikra2 = pathlib.Path(tmp_path)
                        if failoPatikra2.exists() == True :
                          shutil.copy2(tmp_path, (os.path.join(save_path,str(row['Cluster'])+"cluster")) )
                print("\n\nClustered data save path", save_path, "\n\n")

            p1 = save_path + "/" +csv_folder.split("/")[-1].split(".")[0] + "_results.csv"
            print("\n\nClustered csv details saved in", p1 , "\n\n")
            operation_saveCsv(kaipKopija , f"{p1}")

          else:
            print(f"   !!! Alert !!!\n   You need to change N - cluster size [2,{len(X)}]\n   !!! Alert !!!")

        else:
          print(f"   !!! Alert !!!\n   You need to enter more avaible values\n {len(s_copy.columns)}  !!! Alert !!!")

def main():
  print("Welcome to amber clustering - sorting by shape system\n\nPick from avaible operations:\n\n 1 - Reduce size of raw images;")
  print(" 2 - Rename all images in pattern;")
  print(" 3 - Convert raw images to BINARY/Greyscale images;")
  print(" 4 - Create new Data Structure from binary images;")
  print(" 5 - Sort Data Structure CSV, with K-Means;")
  print(" 6 - Display data structure in Correlation table;")
  print(" 7 - Sort Ambers - Data Structure CSV, with SOM;")
  print(" 0 - Exit")

  while True:
      try:
          print("")
          pasirinktaOperacija= int(input("What operation you want to do?\n "))

      except ValueError:
          print("You entered not a number! Try again.")
          continue

      else:
          break 

  while pasirinktaOperacija != 0:
      if pasirinktaOperacija == 1:
            while True:
              try:
                resize_ratio=float(input(f"Enter nuotrauka resize ratio: (ex.: 0.5 - for 2 times smaller nuotrauka, 0.25 - for 4 times smaller nuotrauka )\n"))
                
                if resize_ratio <= 0 and resize_ratio >= 1:
                  print("You entered incorrect ratio")
                  continue

              except ValueError:
                print("You entered not a number! Try again.")
                continue

              else:
                break 
            
            path=str(input(f"Enter aplankalas location from where to read RAW images, or leave empty for default: ( {raw_image_loc} )\n"))
            
            if path == "":
              path = raw_image_loc

            save_path=str(input(f"Enter aplankalas location where to save formatted images, or leave empty for default: ( {smaller_sized_image_loc} )\n"))
            
            if save_path == "":
              save_path = smaller_sized_image_loc
              
            resize_aspect_fit(resize_ratio, path, save_path )

      elif pasirinktaOperacija == 2:

            rename_folder=str(input(f"Enter what aplankalas to rename, leave empty field for default: ( {csv_rename_loc} )\n"))
          
            if rename_folder == "":
                rename_folder = csv_rename_loc    

            renameByRepetition(rename_folder, 3)# pattern)

      elif pasirinktaOperacija == 3:
            aplankalas=str(input(f"Enter nuotrauka aplankalas location or leave empty for default: ( {smaller_sized_image_loc} )\n"))
            
            if aplankalas == "":
              aplankalas = smaller_sized_image_loc

            dvejetainiuNuotraukuVieta=str(input(f"Enter where to save binary images - aplankalas location or leave empty field for default: ( {binary_image_save_location} )\n"))
            
            if dvejetainiuNuotraukuVieta == "":
              dvejetainiuNuotraukuVieta = binary_image_save_location

            operation_1(aplankalas, dvejetainiuNuotraukuVieta )

      elif pasirinktaOperacija == 4:
            global arSektiIrasus
            folder_loacation=str(input(f"Enter binary nuotrauka aplankalas location or leave empty for default: ( {binary_image_save_location} )\n"))
            
            if folder_loacation == "":
              folder_loacation=binary_image_save_location

            csv_save_locat=str(input(f"Enter CSV data structure failoPavadinimas, and file path, or leave empty for default: ( {csv_save_loc_csv} )\n"))
            
            if csv_save_locat == "":
              csv_save_locat=csv_save_loc_csv

            while True:
              try:
                  log_data =int(input("Enter 1 - to print calculation log; 0 - without log. \n "))
              except ValueError:
                  log_data = 0
                  print("You chosen not to print log.")
                  break
                  continue
              else:
                  if log_data != 0:
                    print("You chosen to print log.")
                  else:
                    print("You chosen not to print log.")
                  break 

            operation2(folder_loacation,csv_save_locat, log_data)
            
            arSektiIrasus = 0

      elif pasirinktaOperacija == 5:
            n_clusters = 1
          
            csv_folder=str(input(f"Enter from where to read CSV  (or leave empty field for default: Opt. 1: {csv_save_loc_one} )  ( Opt. 2: {csv_save_loc_group} )\n\
                                                         (Opt. 3: {csv_save_loc_one_orig} )  ( Opt. 4: {csv_save_loc_group_orig} )\n"))
            if csv_folder == "" or csv_folder == "1":
              csv_folder = csv_save_loc_one

            elif  csv_folder == "2" :
              csv_folder = csv_save_loc_group

            elif  csv_folder == "3" :
              csv_folder = csv_save_loc_one_orig

            elif  csv_folder == "4" :
              csv_folder = csv_save_loc_group_orig

            while True:
              try:
                  move_img =int(input("Enter 0 - to copy sorted amber images, or 1 - to do nothing with images. (default 0) \n "))

              except ValueError:
                  move_img = 0
                  print("\n You chosen to copy sorted amber images.")
                  break
                  continue

              else:
                if move_img == 1:
                  print("You chosen to do nothing with images.")

                else:
                  move_img = 0
                  print("\n You chosen to copy sorted amber images.")

                break 

            orig_img_loc=str(input(f"Enter from where to copy images to new location: (Opt. 1: {binary_image_save_location} ) (Opt. 2: {binary_image_save_location_orig} )\n"))
            
            if orig_img_loc == "" or orig_img_loc == "1":
              orig_img_loc = binary_image_save_location

            elif orig_img_loc == "2":
              orig_img_loc = binary_image_save_location_orig

            clusteded_d_folder=str(input(f"Enter where to save new Clustered data: (Opt1. : {sorted_image_loc} ) (Opt2. : {sorted_image_loc_orig} )\n"))
            if clusteded_d_folder == "" or clusteded_d_folder == "1":
              clusteded_d_folder = sorted_image_loc

            elif clusteded_d_folder == "2":
              clusteded_d_folder = sorted_image_loc_orig
            
            model_loc_folder=str(input(f"Enter from where to read Kmean Model or leave empty field for default Opt. 1. ( Opt. 1: {kmean_model_loc} )  (Opt. 2:  )\n")) # K-Mean-MODEL
            if model_loc_folder == "" or model_loc_folder == "1":
              model_loc_folder = kmean_model_loc
            
            while True:
              try:
                  relearn_query =int(input("Enter 0 - to create new k-mean model, or 1 - To reuse existing k-mean model. (default 0) \n "))

              except ValueError:
                  relearn_query = 0
                  print("\n You chosen to create new machine learning model.")
                  break
                  continue
              else:
                  if relearn_query == 1:
                    print("You chosen to reuse existing k-mean machine learning model.")
                  else:
                    relearn_query = 0
                    print("\n You chosen to create new machine learning model.")
                  break 

            operation5_loadcsv_Kmeans(csv_folder, clusteded_d_folder, orig_img_loc, relearn_query, model_loc_folder, move_images = move_img)
      
      elif pasirinktaOperacija == 6:
                
          csv_folder=str(input(f"Enter from where to read CSV  (or leave empty field for default: Opt. 1: {csv_save_loc_one} )  ( Opt. 2: {csv_save_loc_group} )\n\
                                                        (Opt. 3: {csv_save_loc_one_orig} )  ( Opt. 4: {csv_save_loc_group_orig} )\n"))
          if csv_folder == "" or csv_folder == "1":
            csv_folder = csv_save_loc_one

          elif  csv_folder == "2" :
            csv_folder = csv_save_loc_group

          elif  csv_folder == "3" :
            csv_folder = csv_save_loc_one_orig

          elif  csv_folder == "4" :
            csv_folder = csv_save_loc_group_orig

          operation6_datastructure_correlation(csv_folder)

      elif pasirinktaOperacija == 7:
        
            csv_folder=str(input(f"Enter from where to read CSV  (or leave empty field for default: Opt. 1: {csv_save_loc_one} )  ( Opt. 2: {csv_save_loc_group} )\n\
                                                         (Opt. 3: {csv_save_loc_one_orig} )  ( Opt. 4: {csv_save_loc_group_orig} )\n"))
            if csv_folder == "" or csv_folder == "1":
              csv_folder = csv_save_loc_one

            elif  csv_folder == "2" :
              csv_folder = csv_save_loc_group

            elif  csv_folder == "3" :
              csv_folder = csv_save_loc_one_orig

            elif  csv_folder == "4" :
              csv_folder = csv_save_loc_group_orig

            while True:
              try:
                  move_img =int(input("Enter 0 - to copy sorted amber images, or 1 - to do nothing with images. (default 0) \n "))
              except ValueError:
                  move_img = 0
                  print("\n You chosen to copy sorted amber images.")
                  break
                  continue
              else:
                if move_img == 1:
                  print("You chosen to do nothing with images.")
                else:
                  move_img = 0
                  print("\n You chosen to copy sorted amber images.")

                break 

            orig_img_loc=str(input(f"Enter from where to copy images to new location: (Opt. 1: {binary_image_save_location} ) (Opt. 2: {binary_image_save_location_orig} )\n"))
            if orig_img_loc == "" or orig_img_loc == "1":
              orig_img_loc = binary_image_save_location

            elif orig_img_loc == "2":
              orig_img_loc = binary_image_save_location_orig

            clusteded_d_folder=str(input(f"Enter where to save new Clustered data: (Opt1. : {sorted_image_loc} ) (Opt2. : {sorted_image_loc_orig} )\n"))
            if clusteded_d_folder == "" or clusteded_d_folder == "1":
              clusteded_d_folder = sorted_image_loc

            elif clusteded_d_folder == "2":
              clusteded_d_folder = sorted_image_loc_orig
            
            model_loc_folder=str(input(f"Enter from where to read SOM Model or leave empty field for default Opt. 1. ( Opt. 1: {kmean_model_loc} )  (Opt. 2:  )\n")) # K-Mean-MODEL
            if model_loc_folder == "" or model_loc_folder == "1":
              model_loc_folder = kmean_model_loc
            
            while True:
              try:
                  relearn_query =int(input("Enter 0 - to create new SOM model, or 1 - To reuse existing SOM model. (default 0) \n "))
              except ValueError:
                  relearn_query = 0
                  print("\n You chosen to create new machine learning model.")
                  break
                  continue

              else:
                  if relearn_query == 1:
                    print("You chosen to reuse existing k-mean machine learning model.")
                  else:
                    relearn_query = 0
                    print("\n You chosen to create new machine learning model.")
                  break

            operation6_loadcsv_SOM(csv_folder, clusteded_d_folder, orig_img_loc, relearn_query, model_loc_folder, move_images = move_img)

      elif pasirinktaOperacija == 8:
          n_clusters = 1
          csv_folder=str(input(f"Enter from where to read CSV  (or leave empty field for default: Opt. 1: {csv_save_loc_one} )  ( Opt. 2: {csv_save_loc_group} )\n\
                         #                               (Opt. 3: {csv_save_loc_one_orig} )  ( Opt. 4: {csv_save_loc_group_orig} )\n"))
          if csv_folder == "" or csv_folder == "1":
            csv_folder = csv_save_loc_one

          elif  csv_folder == "2" :
            csv_folder = csv_save_loc_group

          elif  csv_folder == "3" :
            csv_folder = csv_save_loc_one_orig

          elif  csv_folder == "4" :
            csv_folder = csv_save_loc_group_orig


          csv_folder2=str(input(f"Enter from where to read CSV  (or leave empty field for default: Opt. 1: {csv_save_loc_one} )  ( Opt. 2: {csv_save_loc_group} )\n\
                             #                           (Opt. 3: {csv_save_loc_one_orig} )  ( Opt. 4: {csv_save_loc_group_orig} )\n"))
          if csv_folder2 == "" or csv_folder2 == "1":
            csv_folder2 = csv_save_loc_one

          elif  csv_folder2 == "2" :
            csv_folder2 = csv_save_loc_group

          elif  csv_folder2 == "3" :
            csv_folder2 = csv_save_loc_one_orig

          elif  csv_folder2 == "4" :
            csv_folder2 = csv_save_loc_group_orig

          while True:
            try:
                move_img =int(1)
            except ValueError:
                move_img = 0
                print("\n You chosen to copy sorted amber images.")
                break
                continue
            else:
              if move_img == 1:
                print("You chosen to do nothing with images.")
              else:
                move_img = 0
                print("\n You chosen to copy sorted amber images.")

              break 

          orig_img_loc=str(1)#input(f"Enter from where to copy images to new location: (Opt. 1: {binary_image_save_location} ) (Opt. 2: {binary_image_save_location_orig} )\n"))
          if orig_img_loc == "" or orig_img_loc == "1":
            orig_img_loc = binary_image_save_location
          elif orig_img_loc == "2":
            orig_img_loc = binary_image_save_location_orig

          clusteded_d_folder=str(1)#input(f"Enter where to save new Clustered data: (Opt1. : {sorted_image_loc} ) (Opt2. : {sorted_image_loc_orig} )\n"))
          if clusteded_d_folder == "" or clusteded_d_folder == "1":
            clusteded_d_folder = sorted_image_loc
          elif clusteded_d_folder == "2":
            clusteded_d_folder = sorted_image_loc_orig
          
          model_loc_folder=str(1)#input(f"Enter from where to read Kmean Model or leave empty field for default Opt. 1. ( Opt. 1: {kmean_model_loc} )  (Opt. 2:  )\n")) # K-Mean-MODEL
          if model_loc_folder == "" or model_loc_folder == "1":
            model_loc_folder = kmean_model_loc
          
          while True:
            try:
                relearn_query =int(0)#input("Enter 0 - to create new k-mean model, or 1 - To reuse existing k-mean model. (default 0) \n "))
            except ValueError:
                relearn_query = 0
                print("\n You chosen to create new machine learning model.")
                break
                continue
            else:
                if relearn_query == 1:
                  print("You chosen to reuse existing k-mean machine learning model.")
                else:
                  relearn_query = 0
                  print("\n You chosen to create new machine learning model.")
                break 

          crossValidation(csv_folder, csv_folder2, clusteded_d_folder, orig_img_loc, relearn_query, model_loc_folder, move_images = move_img)

      print("\n\n 1 - Reduce size of raw images;")
      print(" 2 - Rename all images in pattern;")
      print(" 3 - Convert raw images to BINARY/Greyscale images;")
      print(" 4 - Create new Data Structure from binary images;")
      print(" 5 - Sort Ambers - Data Structure CSV, with K-Means;")
      print(" 6 - Display data structure in Correlation table;")
      print(" 7 - Sort Ambers - Data Structure CSV, with SOM;")
      print(" 0 - Exit")

      time.sleep(1)

      while True:
          try:
              pasirinktaOperacija = int(input("What operation you want to do next?\n  ")) 
          except ValueError:
              print("You entered not a number! Try again.")
              continue
          else:
              break 

  print("Exiting . . .")



def crossValidation(csv_folder, csv_folder2 , cluster_loc = sorted_image_loc ,image_loc = binary_image_save_location, relearn_query = 0,  model_loc_folder = kmean_model_loc, normalization = 0, move_images = 0):
  testi = True
  try:
    amber_csv_data1 = pd.read_csv(csv_folder) 
  except FileNotFoundError:
    testi = False
    print(f"   !!! Alert !!!\n   Wrong CSV location\n   !!! Alert !!!")

  try:
    amber_csv_data2 = pd.read_csv(csv_folder2) 
  except FileNotFoundError:
    testi = False
    print(f"   !!! Alert !!!\n   Wrong CSV location\n   !!! Alert !!!")

  amber_csv_data = pd.concat([amber_csv_data1, amber_csv_data2])

  if not os.path.exists(os.path.dirname(cluster_loc)):
    try:
      os.makedirs(os.path.dirname(cluster_loc))
    except FileNotFoundError:
      print("Wrong Clustered nuotrauka save location")
      testi = False

  if not os.path.exists(os.path.dirname(cluster_loc)):
    try:
      aplankaloSarasas = os.listdir(image_loc)
    except FileNotFoundError:
      print("Wrong original nuotrauka reading location")
      testi = False

  if testi:
    while True:
      try:
          n_klasteriuDydis=int(input("Enter N cluster size for Kmean\n "))
          if n_klasteriuDydis < 2 or n_klasteriuDydis >= len(amber_csv_data):
            print(f" cluster size should be: 2 <= n and n >= {len(amber_csv_data)}")
            continue

      except ValueError:
          print("You entered not a number! Try again.")
          continue

      else:
          break 
    
    amber_csv_data.info()
        
    while True:
      try:
          t1 = str("")#input("Enter DataStruct end limiter ( or leave empty for default colums )\n "))
          if t1 == "" and len(amber_csv_data.columns) <= 23:
            t1 = str("3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19")

          elif t1 == "" and  len(amber_csv_data.columns) > 25:
            t1 = str("3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67")

      except ValueError:
          if t1 == "" and  len(amber_csv_data.columns) <= 23:
            t1 = str("3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19")

          elif t1 == "" and  len(amber_csv_data.columns) > 25:
            t1 = str("3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67")

          else:
            print("You entered not a number! Try again.")
            continue
      else:
          break 
    
    columnsInput = list(map(converter,t1.split()))

    if len(columnsInput) > 1 and amber_csv_data.columns.values.tolist()[columnsInput[0]] != 'CDF':

      map_s = map(converter,t1.split())
      map_s = [x for x in map_s if x >= 0 and x <= len(amber_csv_data.columns)]
      s_copy = amber_csv_data.iloc[:, list(map_s)]
      columnNames = s_copy.columns.values.tolist()

      for dropNames in [string for string in columnNames if 'amber_cnt' in string or 'chaincode' in string or 'amber_picture' in string or 'amber_chain' in string or 'CDF' in string or 'major_axis_coordinates' in string or 'Filename' in string or 'amber_cnt' in string]:
        s_copy = s_copy.drop([dropNames],axis = 1)
      
      if len(s_copy.columns) >= 1:
        s_copy = s_copy.infer_objects()

        kaipKopija = amber_csv_data.copy()

        if relearn_query == 0:
          smin = [];
          smax = [];
          for column in s_copy.columns:
            smin.append(s_copy[column].min())
            smax.append(s_copy[column].max())
            s_copy[column] = (s_copy[column] -  s_copy[column].min() ) / (s_copy[column].max() -  s_copy[column].min())  
          
        elif relearn_query == 1:
          smin = kmean_settings[2]
          smax = kmean_settings[1]
          i = 0
          for column in s_copy.columns:
            s_copy[column] = (s_copy[column] - smin[i]) / (smax[i] - smin[i])    
            i += 1
                  
        X = s_copy.iloc[:, :].values

        if len(X) == 0:
          print(f"   !!! Alert !!!\n   Wrong file or amount of values is too low \n   !!! Alert !!!")

        elif n_klasteriuDydis < len(X) or n_klasteriuDydis > 2:

          scores = []
          cv = KFold(n_splits=4, random_state=42, shuffle=True)
          for train_index, test_index in cv.split(X):
            kaipKopija = amber_csv_data.iloc[test_index[0]:test_index[-1]+1]
            print("Train Index: ", train_index, "\n")
            print("Test Index: ", test_index)

            X_train, X_test = X[train_index], X[test_index]
            joined_dataxy = np.concatenate((X_train, X_test), axis=0)

            amber_csv_data
            ###
            kVidurkiai = KMeans(n_clusters = n_klasteriuDydis)
            x_kmeans_train = kVidurkiai.fit_predict(X_train)
            yKvidurkiai = kVidurkiai.fit_predict(X_test)

            joined_xy = np.append(x_kmeans_train, yKvidurkiai)
           
            ############################################
            # Silhouette 

            print("\n\nThis Silhouette coefficient dimension tells, how points in one cluster are far away from another cluster;", "\nThe silhouette coefficient of whole cluster is the average of all clusters silhouette coefficients, the value can differ between [-1,1];")
            print(" - A value close to coefficient +1 indicates that the clusters are far from each other")
            print(" - A value close to coefficient  0 indicates that the clusters are close to each other and they might overlap")
            print(" - A value close to coefficient -1 indicates that certain points have been assigned incorrectly, which mignt be not for that cluster.\n")
           
            # Apskaiciuojami siluetai sio pavyzdzio

            y_apatinis = 10

           
            siluetoVidurkis = siluetoTaskai(X_test, yKvidurkiai)
            pavyzdzioSiluetoReiksmes = silhouette_samples(X_test, yKvidurkiai)
            means = yKvidurkiai

            print("For n_clusters =", n_klasteriuDydis,
                  "The average silhouette score is: ", siluetoVidurkis," (marked in red dashed line)")

            for i in range(n_klasteriuDydis):
                # Sumuokite pavyzdzius, priklausančius silueto 
                # klasteriam juos surusiuojant

                ith_klasteriopavyzdzioSiluetoReiksmes = \
                pavyzdzioSiluetoReiksmes[means == i]

                ith_klasteriopavyzdzioSiluetoReiksmes.sort()

                klasterioDydis_i = ith_klasteriopavyzdzioSiluetoReiksmes.shape[0]
                y_virsutinis = y_apatinis + klasterioDydis_i

                color = cm.nipy_spectral(float(i) / n_klasteriuDydis)
                plt.fill_betweenx(np.arange(y_apatinis, y_virsutinis),
                                  0, ith_klasteriopavyzdzioSiluetoReiksmes,
                                  facecolor=color, edgecolor=color, alpha=0.7)

                # suzymime klasterius
                plt.text(-0.05, y_apatinis + 0.5 * klasterioDydis_i, str(i))

                # apskaiciuojamas y apatinis kitam plot
                y_apatinis = y_virsutinis + 10 

            plt.xlabel("The silhouette coefficient values")
            plt.ylabel("Cluster label")

            # Vertikali linija vidutiniam visų verčių silueto balui
            plt.axvline(x = siluetoVidurkis, color="red", linestyle="--")

            plt.yticks([]) 
            plt.xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

            plt.suptitle(("Silhouette analysis of KMeans clustering "
                          "with n_clusters = %d" % n_klasteriuDydis),
                        fontsize=15, fontweight='bold')
            plt.show()

            ############################################
            # alkūnės metodas
            if len(X) >= n_klasteriuDydis:
              distortions = []
              if len(X) >= n_klasteriuDydis + n_klasteriuDydis:
                K = range(1, (n_klasteriuDydis + n_klasteriuDydis ))
              else: 
                K = range(1, (n_klasteriuDydis ))

              print("\n\nInertia graph. Which shows how far away are clusters between each other;", "\nIt is suggested to avoid over-clustering and to choose the number of clusters that is close to the rise of the inertia curve;\n")
              distortions = []

              for k in K:
                  kVidurkiuModelis = KMeans(n_clusters = k)
                  
                  kVidurkiuModelis.fit(X_train)
                  kVidurkiuModelis.fit(X_test)

                  distortions.append(kVidurkiuModelis.inertia_)

              plt.figure(figsize=(8,4))
              plt.plot(K, distortions, 'bx-')
              plt.xlabel('k - amount of clusters')
              plt.ylabel('Inertia')
              plt.title('The Elbow Method, which shows the optimal k')
              plt.show()
              
            else:
              print("Cant display Elbow method analysis, the amount of samples is too low")
          ############################################
        else:
          print(f"   !!! Alert !!!\n   You need to change N - cluster size [2,{len(X)}]\n   !!! Alert !!!")
      else:
        print(f"   !!! Alert !!!\n   You need to enter more avaible values\n {len(s_copy.columns)}  !!! Alert !!!")
  


In [ ]:

if __name__ == "__main__":
    main()
    